# PEAR - Parsley's Evolutionary Assessment Ratings

# Update Power Ratings

## Data Prep

In [ ]:
from scipy.optimize import minimize # type: ignore
from scipy.optimize import differential_evolution # type: ignore
from tqdm import tqdm # type: ignore
import matplotlib.pyplot as plt # type: ignore
import requests # type: ignore
from PIL import Image # type: ignore
from io import BytesIO # type: ignore
from matplotlib.lines import Line2D # type: ignore
import cfbd # type: ignore
import pandas as pd # type: ignore
from sklearn.preprocessing import MinMaxScaler # type: ignore
import matplotlib.pyplot as plt # type: ignore
import requests # type: ignore
from PIL import Image # type: ignore
from io import BytesIO # type: ignore
import matplotlib.pyplot as plt # type: ignore
import matplotlib.image as mpimg # type: ignore
import requests # type: ignore
from PIL import Image # type: ignore
from matplotlib.offsetbox import OffsetImage, AnnotationBbox # type: ignore
from matplotlib import gridspec # type: ignore
import os
import math # type: ignore
import matplotlib.patches as patches # type: ignore
from unittest import result
import datetime
import numpy as np # type: ignore
from PIL import ImageGrab # type: ignore
from base64 import b64decode # type: ignore
import PIL # type: ignore
import warnings
import seaborn as sns # type: ignore
from matplotlib.colors import LinearSegmentedColormap # type: ignore
import pytz # type: ignore
import matplotlib.colors as mcolors # type: ignore
import matplotlib.font_manager as fm # type: ignore
checkmark_font = fm.FontProperties(family='DejaVu Sans')
# Suppress all warnings
warnings.filterwarnings("ignore")
np.random.seed(42)

configuration = cfbd.Configuration(
    access_token = '7vGedNNOrnl0NGcSvt92FcVahY602p7IroVBlCA1Tt+WI/dCwtT7Gj5VzmaHrrxS'
)
# configuration = cfbd.Configuration()
# configuration.api_key['Authorization'] = '7vGedNNOrnl0NGcSvt92FcVahY602p7IroVBlCA1Tt+WI/dCwtT7Gj5VzmaHrrxS'
# configuration.api_key_prefix['Authorization'] = 'Bearer'
api_client = cfbd.ApiClient(configuration)
advanced_instance = cfbd.StatsApi(api_client)
games_api = cfbd.GamesApi(api_client)
betting_api = cfbd.BettingApi(api_client)
ratings_api = cfbd.RatingsApi(api_client)
teams_api = cfbd.TeamsApi(api_client)
metrics_api = cfbd.MetricsApi(api_client)
players_api = cfbd.PlayersApi(api_client)
recruiting_api = cfbd.RecruitingApi(api_client)

current_time = datetime.datetime.now(pytz.UTC)
if current_time.month < 6:
    calendar_year = current_time.year - 1
else:
    calendar_year = current_time.year
week_start_list = [*games_api.get_calendar(year = calendar_year)]
calendar_dict = [dict(
    first_game_start = c.first_game_start,
    last_game_start = c.last_game_start,
    season = c.season,
    season_type = c.season_type,
    week = c.week
) for c in week_start_list]
calendar = pd.DataFrame(calendar_dict)
calendar['first_game_start'] = pd.to_datetime(calendar['first_game_start'])
calendar['last_game_start'] = pd.to_datetime(calendar['last_game_start'])
current_year = int(calendar.loc[0, 'season'])

first_game_start = calendar['first_game_start'].iloc[0]
last_game_start = calendar['last_game_start'].iloc[-1]
current_week = None
if current_time < first_game_start:
    current_week = 1
    postseason = False
elif current_time > last_game_start:
    current_week = calendar.iloc[-2, -1] + 1
    postseason = True
else:
    condition_1 = (calendar['first_game_start'] <= current_time) & (calendar['last_game_start'] >= current_time)
    condition_2 = (calendar['last_game_start'].shift(1) < current_time) & (calendar['first_game_start'] > current_time)

    # Combine conditions
    result = calendar[condition_1 | condition_2].reset_index(drop=True)
    if result['season_type'][0] == 'regular':
        current_week = result['week'][0]
        postseason = False
    else:
        current_week = calendar.iloc[-2, -1] + 1
        postseason = True

current_year = int(current_year)
current_week = int(current_week)
print(f"Current Week: {current_week}, Current Year: {current_year}")
print("Double Check The Current Week To Make Sure It Is Correct")

def date_sort(game):
    return game['start_date']

def PEAR_Win_Prob(home_power_rating, away_power_rating):
    return round((1 / (1 + 10 ** ((away_power_rating - (home_power_rating)) / 20.5))) * 100, 2)

def average_team_distribution(num_simulations, schedules, average, team_name):

    def simulate_game_average(win_prob):
        random_outcome = np.random.random() * 100  # Generates a number between 0 and 100
        if random_outcome < win_prob:
            return "W"  # Home team wins, Away team loses
        else:
            return "L"  # Away team wins, Home team loses
        
    def simulate_season_average(schedules, team_name, average):
        wins = 0
        losses = 0
        for _, game in schedules.iterrows():
            if game['home_team'] == team_name:
                opponent_team = game['away_team']
                opponent_pr = game['away_pr']
                win_prob = PEAR_Win_Prob(average, opponent_pr)

                # opponent_elo = game['away_elo']
                # win_prob = round((10**((average-opponent_elo) / 400)) / ((10**((average-opponent_elo) / 400)) + 1)*100, 2)
            else:
                opponent_team = game['home_team']
                opponent_pr = game['home_pr']
                win_prob = 100 - PEAR_Win_Prob(opponent_pr, average)

                # opponent_elo = game['home_elo']
                # win_prob = 100 - round((10**((opponent_elo-average) / 400)) / ((10**((opponent_elo-average) / 400)) + 1)*100, 2)
            
            outcome = simulate_game_average(win_prob)
            if outcome == "W":
                wins += 1
            else:
                losses += 1

        return wins, losses
        
    def monte_carlo_simulation_average(num_simulations, schedules, average, team_name):
        """Runs a Monte Carlo simulation for an average team over multiple seasons."""
        win_results = []
        loss_results = []

        for _ in range(num_simulations):
            wins, losses = simulate_season_average(schedules, team_name, average)
            win_results.append(wins)
            loss_results.append(losses)
        
        return win_results, loss_results

    import statistics
    from collections import Counter
    def analyze_simulation_average(win_results, loss_results, schedules):
        games_played = len(schedules)
        if games_played == 11:
            win_results = [x + .948 for x in win_results]
        elif games_played == 10:
            win_results = [x + (2 * .948) for x in win_results]
    
        avg_wins = statistics.mean(win_results)
        avg_loss = statistics.mean(loss_results)
        most_common_win = statistics.mode(win_results)
        most_common_loss = statistics.mode(loss_results)


        win_counts = Counter(win_results)    
        total_simulations = len(win_results)
        win_percentages = {f"win_{wins}": (win_counts[wins] / total_simulations) for wins in range(13)}
        win_thresholds = pd.DataFrame([win_percentages])
        
        # win_thresholds = {}
        # for wins in range(13):  # 0 to 12 wins
        #     win_thresholds[f'win_{wins}'] = win_df.apply(lambda x: (x == wins).sum() / len(x), axis=0)

        win_thresholds['WIN6%'] = win_thresholds.loc[:, 'win_6':'win_12'].sum(axis=1)
        win_thresholds['expected_wins'] = avg_wins
        win_thresholds['expected_losses'] = avg_loss
        win_thresholds['projected_wins'] = most_common_win
        win_thresholds['projected_losses'] = most_common_loss

        return win_thresholds
    
    avg_win, avg_loss = monte_carlo_simulation_average(num_simulations, schedules, average, team_name)
    win_thresholds = analyze_simulation_average(avg_win, avg_loss,schedules)
    return win_thresholds

if postseason:
    elo_ratings_list = [*ratings_api.get_elo(year=current_year)]
else:
    elo_ratings_list = [*ratings_api.get_elo(year=current_year, week=current_week)]
elo_ratings_dict = [dict(
    team = e.team,
    elo = e.elo
) for e in elo_ratings_list]
elo_ratings = pd.DataFrame(elo_ratings_dict)

# returning production
# production_list = []
# response = players_api.get_returning_production(year = current_year)
# production_list = [*production_list, *response]
# production_dict = [dict(
#     season=r.season,
#     team=r.team,
#     returning_ppa=r.percent_ppa,
#     returning_usage=r.usage
# ) for r in production_list]
# returning_production = pd.DataFrame(production_dict)

# team records
records_list = []
response = games_api.get_records(year=current_year)
records_list = [*records_list, *response]
records_dict = [dict(
    team = r.team,
    games_played = r.total.games,
    wins = r.total.wins,
    losses = r.total.losses,
    conference_games = r.conference_games.games,
    conference_wins = r.conference_games.wins,
    conference_losses = r.conference_games.losses
) for r in records_list]
records = pd.DataFrame(records_dict)

# qb ppa
qb_ppa_list = []
response = metrics_api.get_predicted_points_added_by_player_season(year=current_year, position='QB', threshold=10)
if response:
    qb_ppa_list = [*qb_ppa_list, *response]
    qb_ppa_dict = [dict(
        team=q.team,
        qb_average_ppa=q.average_ppa.all,
        qb_total_ppa=q.total_ppa.all
    ) for q in qb_ppa_list]
    qb_ppa = pd.DataFrame(qb_ppa_dict)
    qb_ppa = qb_ppa.groupby('team', as_index=False).mean()
else:
    qb_ppa = pd.DataFrame(columns=['team', 'qb_average_ppa', 'qb_total_ppa'])

# fpi ranks
team_fpi_list = []
response = ratings_api.get_fpi(year = current_year)
team_fpi_list = [*team_fpi_list, *response]
team_fpi_dict = [dict(
    team = f.team,
    fpi = f.fpi,
    fpi_rank = f.resume_ranks.fpi,
    fpi_sor = f.resume_ranks.strength_of_record,
    fpi_sos = f.resume_ranks.strength_of_schedule,
    def_eff = f.efficiencies.defense,
    off_eff = f.efficiencies.offense,
    special_eff = f.efficiencies.special_teams
) for f in team_fpi_list]
team_fpi = pd.DataFrame(team_fpi_dict)

# srs ranks
team_srs_list = []
response = ratings_api.get_srs(year = current_year)
team_srs_list = [*team_srs_list, *response]
team_srs_dict = [dict(
    team = f.team,
    srs = f.rating,
    srs_rank = f.ranking
) for f in team_srs_list]
team_srs = pd.DataFrame(team_srs_dict).dropna().drop_duplicates()

# sp ranks
team_sp_list = []
response = ratings_api.get_sp(year=current_year)
team_sp_list = [*team_sp_list, *response]
team_sp_dict = [dict(
    team = t.team,
    ranking = t.ranking,
    sp_rating = t.rating
) for t in team_sp_list]
team_sp = pd.DataFrame(team_sp_dict).dropna()

# logo info
logos_info_list = []
response = teams_api.get_teams()
logos_info_list = [*logos_info_list, *response]
logos_info_dict = [dict(
    team = l.school,
    color = l.color,
    alt_color = l.alternate_color,
    logo = l.logos
) for l in logos_info_list]
logos_info = pd.DataFrame(logos_info_dict)
logos_info = logos_info.dropna(subset=['logo', 'color'])

# advanced metrics
advanced_metrics_response = []
response = advanced_instance.get_advanced_season_stats(year = current_year)
advanced_metrics_response = [*advanced_metrics_response, *response]
advanced_metrics = pd.DataFrame()
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
for i in range(len(advanced_metrics_response)):
    data = advanced_metrics_response[i].to_dict() if hasattr(advanced_metrics_response[i], 'to_dict') else vars(advanced_metrics_response[i])

    offense_stats = flatten_dict(data['offense'], parent_key='Offense')
    defense_stats = flatten_dict(data['defense'], parent_key='Defense')
    combined_data = {
        'team':data['team'],
        **offense_stats,
        **defense_stats
    }
    df = pd.DataFrame([combined_data])
    advanced_metrics = pd.concat([advanced_metrics, df], ignore_index=True)
columns_to_keep = ['team', 'Offense_explosiveness', 'Defense_explosiveness', 'Offense_ppa', 'Defense_ppa', 'Defense_havoc_total',
                   'Offense_successRate','Defense_successRate','Offense_powerSuccess','Defense_powerSuccess',
                   'Offense_stuffRate','Defense_stuffRate','Offense_pointsPerOpportunity','Defense_pointsPerOpportunity',
                   'Offense_fieldPosition_averagePredictedPoints','Defense_fieldPosition_averagePredictedPoints',
                   'Offense_fieldPosition_averageStart','Defense_fieldPosition_averageStart']
metrics = advanced_metrics[columns_to_keep]

# conference sp
conference_sp = []
response = ratings_api.get_conference_sp(year=current_year)
conference_sp = [*conference_sp, *response]

sp_conf = [dict(
    conference=c.conference,
    season=c.year,
    sp_conf_rating=c.rating
) for c in conference_sp]
conference_sp_rating = pd.DataFrame(sp_conf)

# team stats
team_stats_list = []
response = advanced_instance.get_team_stats(year=current_year)
team_stats_list = [*team_stats_list, *response]
team_stats_dict = [dict(
    team = s.team,
    stat_name = s.stat_name,
    stat_value = s.stat_value.actual_instance
) for s in team_stats_list]
team_stats = pd.DataFrame(team_stats_dict)
team_stats = team_stats.pivot(index='team', columns='stat_name', values='stat_value').reset_index().fillna(0)
team_stats['total_turnovers'] = team_stats['fumblesRecovered'] + team_stats['passesIntercepted'] - team_stats['turnovers']
team_stats['thirdDownConversionRate'] = round(team_stats['thirdDownConversions'] / team_stats['thirdDowns'],4)
team_stats['fourthDownConversionRate'] = round(team_stats['fourthDownConversions'] / team_stats['fourthDowns'], 4)
team_stats['possessionTimeMinutes'] = round(team_stats['possessionTime'] / 60,2)

# team talent
talent_list = []
for year in range(current_year-3, current_year+1):
    response = teams_api.get_talent(year=year)
    talent_list = [*talent_list, *response]
talent_dict = [dict(
    team=t.team,
    season=t.year,
    talent=t.talent
) for t in talent_list]
talent = pd.DataFrame(talent_dict)

# team info
team_info_list = [*teams_api.get_fbs_teams()]
team_dict = [dict(
    team = t.school,
    conference = t.conference
) for t in team_info_list]
team_info = pd.DataFrame(team_dict)

# team recruiting
recruiting_info_list = []
for year in range(current_year-3, current_year+1):
    response = recruiting_api.get_team_recruiting_rankings(year=year)
    recruiting_info_list = [*recruiting_info_list, *response]
recruiting_info_dict = [dict(
    team = r.team,
    year = r.year,
    points = r.points
) for r in recruiting_info_list]
recruiting = pd.DataFrame(recruiting_info_dict)

print("Data Load Done")

## Data Formatting

In [ ]:
# entire talent profile for the team over the last three years
last_three_rows = talent.groupby('team').tail(3)
avg_talent_per_team = last_three_rows.groupby('team')['talent'].mean().reset_index()
avg_talent_per_team.columns = ['team', 'avg_talent']

# Recruiting points over the last 4 years
last_three = recruiting.groupby('team').tail(3)
recruiting_per_team = last_three.groupby('team')['points'].sum().reset_index()
recruiting_per_team.columns = ['team', 'avg_points']
recruiting_per_team['avg_points'] = recruiting_per_team['avg_points'] + 150

intermediate_1 = pd.merge(team_info, avg_talent_per_team, how='left', on='team')
intermediate_2 = pd.merge(intermediate_1, conference_sp_rating, how='left', on='conference')
intermediate_3 = pd.merge(intermediate_2, team_stats, how='left', on='team')
intermediate_4 = pd.merge(intermediate_3, logos_info, how='left', on='team')
intermediate_5 = pd.merge(intermediate_4, qb_ppa, how='left', on='team')
intermediate_6 = pd.merge(intermediate_5, team_fpi, how='left', on='team')
intermediate_7 = pd.merge(intermediate_6, records, how='left', on='team')
team_data = pd.merge(intermediate_7, metrics, how='left', on='team')

# For military schools and new FBS schools, use recruiting points instead of team talent
# New FBS Schools - you get on here for 3 years
target_teams = ['Air Force', 'Army', 'Navy', 'Kennesaw State', 'Jacksonville State', 'Sam Houston']
mask = team_data['team'].isin(target_teams)
team_data.loc[mask, 'avg_talent'] = team_data.loc[mask, 'team'].map(
    recruiting_per_team.set_index('team')['avg_points']
)
team_data = team_data.drop_duplicates(subset=["team"]).reset_index(drop=True)

print("Data Formatting Done")
print("Starting Optimization")

## Optimization of Ratings

In [ ]:
############ IN HOUSE PR #################

# All the scalers used for the team data
scaler100 = MinMaxScaler(feature_range=(1, 100))
scaler60 = MinMaxScaler(feature_range=(40,98.8))
scaler10 = MinMaxScaler(feature_range=(1,10))
scalerTurnovers = MinMaxScaler(feature_range=(1, 100))
scalerPenalties = MinMaxScaler(feature_range=(1, 100))
scalerThirdDown = MinMaxScaler(feature_range=(1, 100))
scalerTalent = MinMaxScaler(feature_range=(100,1000))
scalerAvgFieldPosition = MinMaxScaler(feature_range=(-10,10))
scalerPPO = MinMaxScaler(feature_range=(1,100))

#################################################################################################################################################

# scaling all the data based on the scaler
team_data['sp_conf_scaled'] = scaler10.fit_transform(team_data[['sp_conf_rating']])
team_data['total_turnovers_scaled'] = scalerTurnovers.fit_transform(team_data[['total_turnovers']])
team_data['possession_scaled'] = scaler100.fit_transform(team_data[['possessionTimeMinutes']])
team_data['third_down_scaled'] = scalerThirdDown.fit_transform(team_data[['thirdDownConversionRate']])
team_data['offense_avg_field_position_scaled'] = -1*scalerAvgFieldPosition.fit_transform(team_data[['Offense_fieldPosition_averageStart']])
team_data['defense_avg_field_position_scaled'] = scalerAvgFieldPosition.fit_transform(team_data[['Defense_fieldPosition_averageStart']])
team_data['offense_ppo_scaled'] = scalerPPO.fit_transform(team_data[['Offense_pointsPerOpportunity']])
team_data['offense_success_scaled'] = scaler100.fit_transform(team_data[['Offense_successRate']])
team_data['offense_explosive'] = scaler100.fit_transform(team_data[['Offense_explosiveness']])
team_data['talent_scaled'] = scalerTalent.fit_transform(team_data[['avg_talent']])

def_ppo_min = team_data['Defense_pointsPerOpportunity'].min()
def_ppo_max = team_data['Defense_pointsPerOpportunity'].max()
team_data['defense_ppo_scaled'] = 100 - (team_data['Defense_pointsPerOpportunity'] - def_ppo_min) * 99 / (def_ppo_max - def_ppo_min)

pen_min = team_data['penaltyYards'].min()
pen_max = team_data['penaltyYards'].max()
team_data['penalties_scaled'] = 100 - (team_data['penaltyYards'] - pen_min) * 99 / (pen_max - pen_min)

off_field_min = team_data['Offense_fieldPosition_averageStart'].min()
off_field_max = team_data['Offense_fieldPosition_averageStart'].max()
team_data['offense_avg_field_position_scaled'] = 100 - (team_data['Offense_fieldPosition_averageStart'] - off_field_min) * 99 / (off_field_max - off_field_min)

team_data['offense_ppa_scaled'] = scaler100.fit_transform(team_data[['Offense_ppa']])
ppa_min = team_data['Defense_ppa'].min()
ppa_max = team_data['Defense_ppa'].max()
team_data['defense_ppa_scaled'] = 100 - (team_data['Defense_ppa'] - ppa_min) * 99 / (ppa_max - ppa_min)

success_min = team_data['Defense_successRate'].min()
success_max = team_data['Defense_successRate'].max()
team_data['defense_success_scaled'] = 100 - (team_data['Defense_successRate'] - success_min) * 99 / (success_max - success_min)

explosiveness_min = team_data['Defense_explosiveness'].min()
explosiveness_max = team_data['Defense_explosiveness'].max()
team_data['defense_explosive'] = 100 - (team_data['Defense_explosiveness'] - explosiveness_min) * 99 / (explosiveness_max - explosiveness_min)

#################################################################################################################################################

# calculating the adjusted metric as well as the power rating for each team
alpha = .05
team_data['adjusted_metric'] = (0.7 * (team_data['offense_success_scaled'] + team_data['defense_success_scaled']) +
                                (alpha * team_data['sp_conf_scaled']**0.5) +
                                0.25 * (team_data['offense_explosive'] + team_data['defense_explosive']) +
                                (0*team_data['talent_scaled']) + (0.4*(team_data['total_turnovers_scaled'] + team_data['penalties_scaled'] + team_data['offense_ppo_scaled'])))
team_data['average_metric'] = (team_data['offense_success_scaled'] + team_data['offense_explosive'] + team_data['offense_ppa_scaled'] + 
                            team_data['defense_success_scaled'] + team_data['defense_explosive'] + team_data['defense_ppa_scaled']) / 6
team_data['in_house_pr'] = scaler60.fit_transform(team_data[['adjusted_metric']]).round(2)
team_data['in_house_pr'] = round(team_data['in_house_pr'] - team_data['in_house_pr'].mean(), 1)

###############################################################################
###############################################################################
###############################################################################
## When adding in the pre-season ratings, I think this is a good spot for it ##
###############################################################################
###############################################################################
###############################################################################

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import differential_evolution

# --- SETUP COLUMN GROUPS ---
offensive_columns = ['Offense_successRate', 'Offense_explosiveness', 'Offense_ppa','Offense_pointsPerOpportunity']
defensive_columns = ['Defense_successRate', 'Defense_explosiveness', 'Defense_ppa','Defense_pointsPerOpportunity']
features_all = offensive_columns + defensive_columns + ['avg_talent', 'thirdDownConversionRate', 'total_turnovers', 'in_house_pr']

# --- SCALING FUNCTION ---
def scale_columns(df, columns, reverse=False):
    scaler = MinMaxScaler(feature_range=(1, 100))
    data = -df[columns] if reverse else df[columns]
    scaled = scaler.fit_transform(data)
    return pd.DataFrame(scaled, columns=columns)

# --- APPLY SCALING ---
scaling_config = {
    'offense': (offensive_columns, False),
    'defense': (defensive_columns, True),
    'talent': (['avg_talent'], False),
    'third_down': (['thirdDownConversionRate'], False),
    'fourth_down': (['fourthDownConversionRate'], False),
    'turnovers': (['total_turnovers'], True),
    'returns': (['puntReturnTDs', 'kickReturnTDs'], False),
    'havoc': (['Defense_havoc_total'], False),
}
for _, (cols, rev) in scaling_config.items():
    team_data[cols] = scale_columns(team_data, cols, reverse=rev)

# --- MERGE DATA ---
merged_data = pd.merge(team_data, team_sp[['team', 'ranking']], on='team')
X = merged_data[features_all].values  # Feature matrix

# --- OBJECTIVE FUNCTION ---
def objective_spearman(weights_all):
    feature_weights = np.array(weights_all[:-2])
    rank_weights = np.array(weights_all[-2:])
    
    if np.sum(feature_weights) == 0 or np.all(feature_weights < 0.01):
        return 1
    
    feature_weights /= np.sum(feature_weights)
    rank_weights /= np.sum(rank_weights)

    power_ranking = X @ feature_weights
    merged_data['power_ranking'] = power_ranking
    merged_data['calculated_rank'] = merged_data['power_ranking'].rank(ascending=False)

    merged_data['combined_rank'] = (
        merged_data['fpi_rank'] * rank_weights[0] +
        merged_data['ranking'] * rank_weights[1]
    )

    corr = merged_data[['calculated_rank', 'combined_rank']].corr(method='spearman').iloc[0, 1]
    return -corr

# --- RUN DIFFERENTIAL EVOLUTION ---
def run_differential_evolution():
    bounds = [(0, 1)] * (len(features_all) + 2)
    result = differential_evolution(
        objective_spearman,
        bounds=bounds,
        strategy='best1bin',
        maxiter=1000,
        tol=1e-4,
        polish=True,
        seed=42
    )

    all_weights = result.x
    feature_weights = all_weights[:-2]
    rank_weights = all_weights[-2:]

    feature_weights /= np.sum(feature_weights)
    rank_weights /= np.sum(rank_weights)

    selected_features = [feat for feat, w in zip(features_all, feature_weights) if w > 0.01]
    selected_weights = [w for w in feature_weights if w > 0.01]

    return selected_features, selected_weights, rank_weights, -result.fun

# --- GET BEST FEATURES & WEIGHTS ---
selected_features, best_weights, rank_weights, best_spearman = run_differential_evolution()

feature_weights_dict = dict(zip(selected_features, best_weights))
sorted_features_weights = sorted(feature_weights_dict.items(), key=lambda item: item[1], reverse=True)

print("Selected Features:")
for feature, weight in sorted_features_weights:
    print(f"{feature}: {weight:.4f}")
merged_data['combined_rank'] = merged_data['fpi_rank'] * rank_weights[0] + merged_data['ranking'] * rank_weights[1]

# --- FINAL POWER RANKING COLUMN ---
merged_data['power_ranking'] = sum(
    merged_data[feat] * weight for feat, weight in zip(selected_features, best_weights)
)
output_model = merged_data.copy()
team_data = output_model.copy()

## Formatting of Output

In [ ]:
######################################## TEAM STATS AND RANKINGS #################################################

team_data['PBR'] = team_data['penaltyYards'] / team_data['talent_scaled']
team_data['PBR_rank'] = team_data['PBR'].rank(method='min', ascending=True)

team_data['STM'] = (
    (team_data['kickReturnYards'] / team_data['kickReturns']) +
    (team_data['puntReturnYards'] / team_data['puntReturns']) -
    team_data['Offense_fieldPosition_averageStart'] +
    team_data['Defense_fieldPosition_averageStart']
)
team_data['STM_rank'] = team_data['STM'].rank(method='min', ascending=False)

team_data['DCE'] = (
    (team_data['possessionTimeMinutes'] / team_data['games_played']) +
    (10 * team_data['thirdDownConversionRate']) +
    (20 * team_data['fourthDownConversionRate'])
)
team_data['DCE_rank'] = team_data['DCE'].rank(method='min', ascending=False)

team_data['DefensivePossessionTime'] = (team_data['games_played'] * 60) - team_data['possessionTimeMinutes']
team_data['DDE'] = ( 
    (0.6 * team_data['tacklesForLoss']) +
    (4 * team_data['interceptions']) +
    (6 * team_data['fumblesRecovered']) +
    (1.6 * team_data['sacks'])
)
team_data['DDE_rank'] = team_data['DDE'].rank(method='min', ascending=False)

team_data["offensive_total"] = team_data[offensive_columns].sum(axis=1)
team_data["offensive_rank"] = team_data["offensive_total"].rank(ascending=False, method="dense").astype(int)
team_data["defensive_total"] = team_data[defensive_columns].sum(axis=1)
team_data["defensive_rank"] = team_data["defensive_total"].rank(ascending=False, method="dense").astype(int)

team_data['talent_scaled_rank'] = team_data['talent_scaled'].rank(method='min', ascending=False)
team_data['offense_success_rank'] = team_data['offense_success_scaled'].rank(method='min', ascending=False)
team_data['defense_success_rank'] = team_data['defense_success_scaled'].rank(method='min', ascending=False)
team_data['offense_explosive_rank'] = team_data['offense_explosive'].rank(method='min', ascending=False)
team_data['defense_explosive_rank'] = team_data['defense_explosive'].rank(method='min', ascending=False)
team_data['total_turnovers_rank'] = team_data['total_turnovers_scaled'].rank(method='min', ascending=False)
team_data['penalties_rank'] = team_data['penalties_scaled'].rank(method='min', ascending=False)

######################################## POWER RATING #################################################

team_data['power_rating'] = team_data['power_ranking'] - team_data['power_ranking'].mean()
current_range = team_data['power_rating'].max() - team_data['power_rating'].min()
desired_range = 50  # The target range
scaling_factor = desired_range / current_range
team_data['power_rating'] = round(team_data['power_rating'] * scaling_factor,2)

######################################## FINAL FORMATTING #################################################

team_data = team_data.sort_values(by='power_rating', ascending=False).reset_index(drop=True)
team_data['power_rating'] = round(team_data['power_rating'], 1)
team_data = team_data.drop_duplicates(subset='team')
team_power_rankings = team_data[['team', 'power_rating', 'conference']]
team_power_rankings = team_power_rankings.sort_values(by='power_rating', ascending=False).reset_index(drop=True)

#### if top team is too high, use this
# team_power_rankings.iloc[0,1] = round(team_power_rankings.iloc[1,1] + round((team_power_rankings.iloc[0,1] - team_power_rankings.iloc[1,1]) / 2, 1),1)
# team_data.loc[0, 'power_rating'] = team_power_rankings.iloc[0, 1]

team_power_rankings.index = team_power_rankings.index + 1
team_power_rankings['week'] = current_week
team_power_rankings['year'] = current_year

from joblib import Parallel, delayed
from collections import defaultdict
import numpy as np
import pandas as pd

# --- Vectorized PEAR Win Prob function ---
def PEAR_Win_Prob_vectorized(home_pr, away_pr):
    rating_diff = np.array(home_pr) - np.array(away_pr)
    return np.round(1 / (1 + 10 ** (-rating_diff / 7.5)) * 100, 2)

# --- Get games and build base schedule ---
start_week = 1
end_week = 17
games_list = []
for week in range(start_week, end_week):
    response = games_api.get_games(year=current_year, week=week, classification='fbs')
    games_list.extend(response)
if postseason:
    response = games_api.get_games(year=current_year, classification='fbs', season_type='postseason')
    games_list.extend(response)

games = [
    dict(
        id=g.id, season=g.season, week=g.week, start_date=g.start_date,
        home_team=g.home_team, home_elo=g.home_pregame_elo,
        away_team=g.away_team, away_elo=g.away_pregame_elo,
        home_points=g.home_points, away_points=g.away_points,
        neutral=g.neutral_site
    )
    for g in games_list if g.home_pregame_elo is not None and g.away_pregame_elo is not None
]

games.sort(key=date_sort)
year_long_schedule = pd.DataFrame(games)

# --- Merge power ratings ---
year_long_schedule = year_long_schedule.merge(team_data[['team', 'power_rating']],
                                              left_on='home_team', right_on='team', how='left')\
                                       .rename(columns={'power_rating': 'home_pr'}).drop(columns='team')
year_long_schedule = year_long_schedule.merge(team_data[['team', 'power_rating']],
                                              left_on='away_team', right_on='team', how='left')\
                                       .rename(columns={'power_rating': 'away_pr'}).drop(columns='team')

# --- Add Win Probabilities ---
year_long_schedule['PEAR_win_prob'] = PEAR_Win_Prob_vectorized(
    year_long_schedule['home_pr'], year_long_schedule['away_pr']
)

year_long_schedule['home_win_prob'] = np.round(
    (10 ** ((year_long_schedule['home_elo'] - year_long_schedule['away_elo']) / 400)) /
    ((10 ** ((year_long_schedule['home_elo'] - year_long_schedule['away_elo']) / 400)) + 1) * 100, 2
)

# --- Pre-index schedules ---
team_schedules = {team: year_long_schedule[(year_long_schedule['home_team'] == team) | 
                                           (year_long_schedule['away_team'] == team)]
                  for team in team_data['team']}

# --- SOS Calculation ---
average_pr = round(team_data['power_rating'].mean(), 2)
good_team_pr = round(team_data['power_rating'].std() + average_pr, 2)
elite_team_pr = round(2 * team_data['power_rating'].std() + average_pr, 2)

def calc_expected(team):
    schedule = team_schedules[team]
    df = average_team_distribution(1000, schedule, elite_team_pr, team)
    return df['expected_wins'].values[0]

expected_wins_list = Parallel(n_jobs=-1)(delayed(calc_expected)(team) for team in team_data['team'])

SOS = pd.DataFrame({'team': team_data['team'], 'avg_expected_wins': expected_wins_list})
SOS = SOS.sort_values('avg_expected_wins').reset_index(drop=True)
SOS['SOS'] = SOS.index + 1
print("SOS Calculation Done")

# --- SOR Calculation ---
completed_games = year_long_schedule[year_long_schedule['home_points'].notna()].copy()

def calc_sor(team):
    games_played = records.loc[records['team'] == team, 'games_played'].values[0]
    wins = records.loc[records['team'] == team, 'wins'].values[0]
    team_games = completed_games[(completed_games['home_team'] == team) | 
                                 (completed_games['away_team'] == team)].copy()

    home_probs = PEAR_Win_Prob_vectorized(good_team_pr, team_games['away_pr'])
    away_probs = 100 - PEAR_Win_Prob_vectorized(team_games['home_pr'], good_team_pr)

    team_games['good_win_prob'] = np.where(team_games['home_team'] == team, home_probs, away_probs)

    home_probs = PEAR_Win_Prob_vectorized(average_pr, team_games['away_pr'])
    away_probs = 100 - PEAR_Win_Prob_vectorized(team_games['home_pr'], average_pr)
    team_games['avg_win_prob'] = np.where(team_games['home_team'] == team, home_probs, away_probs)

    home_probs = PEAR_Win_Prob_vectorized(elite_team_pr, team_games['away_pr'])
    away_probs = 100 - PEAR_Win_Prob_vectorized(team_games['home_pr'], elite_team_pr)
    team_games['elite_win_prob'] = np.where(team_games['home_team'] == team, home_probs, away_probs)

    current_xWins = round(team_games['avg_win_prob'].sum() / 100, 2)
    good_xWins = round(team_games['good_win_prob'].sum() / 100, 2)
    elite_xWins = round(team_games['elite_win_prob'].sum() / 100, 2)

    if games_played != len(team_games):
        current_xWins += 1
        good_xWins += 1
        elite_xWins += 1

    return round(wins - current_xWins, 2), round(wins - good_xWins, 2), round(wins - elite_xWins, 2)

sor_results = Parallel(n_jobs=-1)(delayed(calc_sor)(team) for team in team_data['team'])
SOR = pd.DataFrame(sor_results, columns=['wins_above_average','wins_above_good','wins_above_elite'])
SOR.insert(0, 'team', team_data['team'])
SOR = SOR.sort_values('wins_above_good', ascending=False).reset_index(drop=True)
SOR['SOR'] = SOR.index + 1
print("SOR Calculation Done")

# --- Most Deserving Calculation ---
num_12_pr = team_data['power_rating'].iloc[11]

def f(mov):
    return np.clip(np.log(np.abs(mov) + 1) * np.sign(mov), -10, 10)

completed_games['margin_of_victory'] = completed_games['home_points'] - completed_games['away_points']

def calc_deserving(team):
    games_played = records.loc[records['team'] == team, 'games_played'].values[0]
    wins = records.loc[records['team'] == team, 'wins'].values[0]
    team_games = completed_games[(completed_games['home_team'] == team) |
                                 (completed_games['away_team'] == team)].copy()

    mov_adj = f(team_games['margin_of_victory'])

    home_probs = PEAR_Win_Prob_vectorized(num_12_pr, team_games['away_pr']) + mov_adj
    away_probs = 100 - PEAR_Win_Prob_vectorized(team_games['home_pr'], num_12_pr) - mov_adj
    team_games['adj_win_prob'] = np.where(team_games['home_team'] == team, home_probs, away_probs)

    xWins = round(team_games['adj_win_prob'].sum() / 100, 3)
    if games_played != len(team_games):
        xWins += 1
    return round(wins - xWins, 3)

deserving_results = Parallel(n_jobs=-1)(delayed(calc_deserving)(team) for team in team_data['team'])
most_deserving = pd.DataFrame({'team': team_data['team'], 'most_deserving_wins': deserving_results})
most_deserving = most_deserving.sort_values('most_deserving_wins', ascending=False).reset_index(drop=True)
most_deserving['most_deserving'] = most_deserving.index + 1
print("Most Deserving Calculation Done")

team_data = pd.merge(team_data, SOS, how='left', on='team')
team_data = pd.merge(team_data, SOR, how='left', on='team')
team_data = pd.merge(team_data, most_deserving, how='left', on='team')
team_data = pd.merge(team_data, elo_ratings, how='left', on='team')

## Saving Output

In [ ]:
folder_path = f"./PEAR/PEAR Football/y{current_year}/Data"
os.makedirs(folder_path, exist_ok=True)

folder_path = f"./PEAR/PEAR Football/y{current_year}/Ratings"
os.makedirs(folder_path, exist_ok=True)

folder_path = f"./PEAR/PEAR Football/y{current_year}/Spreads"
os.makedirs(folder_path, exist_ok=True)

team_data.to_csv(f"./PEAR/PEAR Football/y{current_year}/Data/team_data_week{current_week}_no_adjustments.csv")
team_power_rankings.to_csv(f'./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{current_week}_no_adjustments.csv')

# Visual Content

## Prep Work

In [ ]:
from scipy.optimize import minimize # type: ignore
from scipy.optimize import differential_evolution # type: ignore
from tqdm import tqdm # type: ignore
import matplotlib.pyplot as plt # type: ignore
import requests # type: ignore
from PIL import Image # type: ignore
from scipy.stats import norm # type: ignore
import requests
from PIL import Image
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed # type: ignore
from io import BytesIO # type: ignore
from matplotlib.lines import Line2D # type: ignore
import cfbd # type: ignore
import pandas as pd # type: ignore
from sklearn.preprocessing import MinMaxScaler # type: ignore
import matplotlib.pyplot as plt # type: ignore
import requests # type: ignore
from PIL import Image # type: ignore
from io import BytesIO # type: ignore
import matplotlib.pyplot as plt # type: ignore
import matplotlib.image as mpimg # type: ignore
import requests # type: ignore
from PIL import Image # type: ignore
from matplotlib.offsetbox import OffsetImage, AnnotationBbox # type: ignore
from matplotlib import gridspec # type: ignore
import os
import math # type: ignore
import matplotlib.patches as patches # type: ignore
from unittest import result
import datetime
import numpy as np # type: ignore
from PIL import ImageGrab # type: ignore
from base64 import b64decode # type: ignore
import PIL # type: ignore
import warnings
import seaborn as sns # type: ignore
from matplotlib.colors import LinearSegmentedColormap # type: ignore
import pytz # type: ignore
from datetime import datetime
import matplotlib.colors as mcolors # type: ignore
import matplotlib.font_manager as fm # type: ignore
from IPython.core.magic import register_cell_magic
checkmark_font = fm.FontProperties(family='DejaVu Sans')
# Suppress all warnings
warnings.filterwarnings("ignore")
np.random.seed(42)
configuration = cfbd.Configuration(
    access_token = '7vGedNNOrnl0NGcSvt92FcVahY602p7IroVBlCA1Tt+WI/dCwtT7Gj5VzmaHrrxS'
)
api_client = cfbd.ApiClient(configuration)
advanced_instance = cfbd.StatsApi(api_client)
games_api = cfbd.GamesApi(api_client)
betting_api = cfbd.BettingApi(api_client)
ratings_api = cfbd.RatingsApi(api_client)
teams_api = cfbd.TeamsApi(api_client)
metrics_api = cfbd.MetricsApi(api_client)
players_api = cfbd.PlayersApi(api_client)
recruiting_api = cfbd.RecruitingApi(api_client)

week_start_list = [*games_api.get_calendar(year = 2025)]
calendar_dict = [dict(
    first_game_start = c.first_game_start,
    last_game_start = c.last_game_start,
    season = c.season,
    season_type = c.season_type,
    week = c.week
) for c in week_start_list]
calendar = pd.DataFrame(calendar_dict)
calendar['first_game_start'] = pd.to_datetime(calendar['first_game_start'])
calendar['last_game_start'] = pd.to_datetime(calendar['last_game_start'])
current_year = int(calendar.loc[0, 'season'])
first_game_start = calendar['first_game_start'].iloc[0]
last_game_start = calendar['last_game_start'].iloc[-1]

current_week = None
current_time = datetime.now(pytz.UTC)
if current_time < first_game_start:
    current_week = 1
    postseason = False
elif current_time > last_game_start:
    current_week = calendar.iloc[-2, -1] + 1
    postseason = True
else:
    condition_1 = (calendar['first_game_start'] <= current_time) & (calendar['last_game_start'] >= current_time)
    condition_2 = (calendar['last_game_start'].shift(1) < current_time) & (calendar['first_game_start'] > current_time)

    # Combine conditions
    result = calendar[condition_1 | condition_2].reset_index(drop=True)
    if result['season_type'][0] == 'regular':
        current_week = result['week'][0]
        postseason = False
    else:
        current_week = calendar.iloc[-2, -1] + 1
        postseason = True
        
logos_info_list = []
response = teams_api.get_teams()
logos_info_list = [*logos_info_list, *response]
logos_info_dict = [dict(
    team = l.school,
    color = l.color,
    alt_color = l.alternate_color,
    logo = l.logos
) for l in logos_info_list]
logos = pd.DataFrame(logos_info_dict)
logos = logos.dropna(subset=['logo', 'color'])

print(current_week, current_year)


############### NEED TO UPDATE ###############
start_season_data = pd.read_csv(f"./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{current_week}.csv")
if os.path.exists(f"./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{current_week-1}.csv"):
    last_week_data = pd.read_csv(f"./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{current_week-1}.csv")
else:
    last_week_data = pd.read_csv(f"./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{current_week}.csv")
week_to_check = current_week - 4
last_month_data = None
while week_to_check <= current_week:
    file_path = f"./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{week_to_check}.csv"
    if os.path.exists(file_path):
        # If the file is found, read it in and break the loop
        last_month_data = pd.read_csv(file_path)
        break    
    week_to_check += 1

all_data = pd.read_csv(f"./PEAR/PEAR Football/y{current_year}/Data/team_data_week{current_week}.csv")
team_data = pd.read_csv(f'./PEAR/PEAR Football/y{current_year}/Ratings/PEAR_week{current_week}.csv')

1 2025


In [ ]:
def date_sort(game):
    game_date = datetime.datetime.strptime(game['start_date'], "%Y-%m-%dT%H:%M:%S.000Z")
    return game_date

def PEAR_Win_Prob(home_power_rating, away_power_rating):
    return round((1 / (1 + 10 ** ((away_power_rating - (home_power_rating)) / 20.5))) * 100, 2)
    
games = []
for week in range(1,current_week):
    response = games_api.get_games(year=current_year, week=week,division = 'fbs')
    games = [*games, *response]
games = [dict(
            id=g.id,
            season=g.season,
            week=g.week,
            start_date=g.start_date,
            home_team=g.home_team,
            home_elo=g.home_pregame_elo,
            away_team=g.away_team,
            away_elo=g.away_pregame_elo,
            home_points = g.home_points,
            away_points = g.away_points
            ) for g in games if g.home_pregame_elo is not None and g.away_pregame_elo is not None]
games.sort(key=date_sort)
schedule_info = pd.DataFrame(games)

schedule_info = schedule_info.merge(team_data[['team', 'power_rating']], 
                                    left_on='home_team', 
                                    right_on='team', 
                                    how='left').rename(columns={'power_rating': 'home_pr'})
schedule_info = schedule_info.drop(columns=['team'])
schedule_info = schedule_info.merge(team_data[['team', 'power_rating']], 
                                    left_on='away_team', 
                                    right_on='team', 
                                    how='left').rename(columns={'power_rating': 'away_pr'})
schedule_info = schedule_info.drop(columns=['team'])

# Apply the PEAR_Win_Prob function to the schedule_info DataFrame
schedule_info['PEAR_win_prob'] = schedule_info.apply(
    lambda row: PEAR_Win_Prob(row['home_pr'], row['away_pr']), axis=1
)

# Elo Win Probability
schedule_info['home_win_prob'] = round((10**((schedule_info['home_elo'] - schedule_info['away_elo']) / 400)) / ((10**((schedule_info['home_elo'] - schedule_info['away_elo']) / 400)) + 1)*100,2)

if postseason:
    elo_ratings_list = [*ratings_api.get_elo(year=current_year)]
else:
    elo_ratings_list = [*ratings_api.get_elo(year=current_year, week=current_week)]
elo_ratings_dict = [dict(
    team = e.team,
    elo = e.elo
) for e in elo_ratings_list]
elo_ratings = pd.DataFrame(elo_ratings_dict)

records_list = []
response = games_api.get_records(year=current_year)
records_list = [*records_list, *response]
records_dict = [dict(
    team = r.team,
    games_played = r.total.games,
    wins = r.total.wins,
    losses = r.total.losses,
    conference_games = r.conference_games.games,
    conference_wins = r.conference_games.wins,
    conference_losses = r.conference_games.losses
) for r in records_list]
records = pd.DataFrame(records_dict)

@register_cell_magic
def capture_png(line, cell):
    get_ipython().run_cell_magic(
        'capture',
        ' --no-stderr --no-stdout result',
        cell
    )
    out_paths = line.strip().split(' ')
    for output in result.outputs:
        data = output.data
        if 'image/png' in data:
            path = out_paths.pop(0)
            if not path:
                raise ValueError('Too few paths given!')
            png_bytes = data['image/png']
            if isinstance(png_bytes, str):
                png_bytes = b64decode(png_bytes)
            assert isinstance(png_bytes, bytes)
            bytes_io = BytesIO(png_bytes)
            image = PIL.Image.open(bytes_io)
            image.save(path, 'png')

## Function Loads

In [ ]:
def best_and_worst(all_data, logos, metric, title, subtitle):
    if metric == 'average_metric_rank':
        top_25_best = all_data.sort_values(metric, ascending=True)[:25].reset_index(drop=True)
        top_25_worst = all_data.sort_values(metric, ascending=False)[:25].reset_index(drop=True)
        top_25_worst = top_25_worst.sort_values(metric, ascending=True)[:25].reset_index(drop=True)
    else:
        top_25_best = all_data.sort_values(metric, ascending=False)[:25].reset_index(drop=True)
        top_25_worst = all_data.sort_values(metric, ascending=True)[:25].reset_index(drop=True)
        top_25_worst = top_25_worst.sort_values(metric, ascending=False)[:25].reset_index(drop=True)

    # Create a figure with 5 rows and 10 columns
    fig, axs = plt.subplots(5, 10, figsize=(20, 10), dpi=125)

    # Adjust space between subplots
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    fig.patch.set_facecolor('#CECEB2')

    # Title and description
    plt.suptitle(title, fontsize=20, fontweight='bold', color='black')
    plt.text(0.505, 0.935, subtitle, ha='center', fontsize=14, color='black', transform=fig.transFigure)

    # Fill the grid alternating Best and Worst Defenses
    for i in range(5):  # There are 5 rows
        # Best defenses: Columns 0-4 for each row (Best in every odd index)
        for j in range(5):  
            ax = axs[i, j]
            team = top_25_best.loc[i*5 + j, 'team']
            logo_url = logos[logos['team'] == team]['logo'].values[0][0]
            response = requests.get(logo_url)
            img = Image.open(BytesIO(response.content))
            ax.imshow(img)
            ax.set_facecolor('#f0f0f0')
            # ax.set_title(f"#{i*5 + j + 1} {team} \n{round(top_25_best.loc[i*5 + j, metric], 1)}", fontsize=8, fontweight='bold')
            ax.set_title(f"#{i*5 + j + 1} \n{round(top_25_best.loc[i*5 + j, metric], 1)}", fontsize=14, fontweight='bold')
            ax.axis('off')

        # Worst defenses: Columns 5-9 for each row (Worst in every even index after 5)
        for j in range(5, 10):  
            ax = axs[i, j]
            team = top_25_worst.loc[i*5 + (j-5), 'team']
            logo_url = logos[logos['team'] == team]['logo'].values[0][0]
            response = requests.get(logo_url)
            img = Image.open(BytesIO(response.content))
            ax.imshow(img)
            ax.set_facecolor('#f0f0f0')
            
            # Start counting for Worst from 134 and decrement
            worst_rank = (len(all_data) - 24) + (i*5 + (j-5)) 
            ax.set_title(f"#{worst_rank} \n{round(top_25_worst.loc[i*5 + (j-5), metric], 1)}", fontsize=14, fontweight='bold')
            ax.axis('off')

    fig.add_artist(Line2D([0.512, 0.512], [0.12, 0.92], color='black', lw=5))
    fig.text(0.13, 0.96, "Best", ha='left', va='center', fontsize=20, fontweight='bold', color='black')
    fig.text(0.89, 0.96, "Worst", ha='right', va='center', fontsize=20, fontweight='bold', color='black')

    # Show the final figure
    plt.show()

# Function to plot the bar chart for a given dataset
def plot_bar_charts(ax, data, logos, metric, title, top_or_bottom):
    if top_or_bottom == 'Top':
        top_or_bottom = True
    else:
        top_or_bottom = False
    # Data
    teams = data['team']
    scores = data[metric]
    
    team_colors = {row['team']: row['color'] for _, row in logos.iterrows()}
    colors = [team_colors[team] for team in teams]

    bar_spacing = 5  # This factor increases the spacing between the bars
    y_pos = range(len(teams))
    y_pos = [y * bar_spacing for y in y_pos]

    # Plot bars
    bars = ax.barh(y_pos, scores, color=colors, align='center', height=4)

    for i, (y, team, bar) in enumerate(zip(y_pos, teams, bars)):
        logo_url = logos[logos['team'] == team]['logo'].values[0][0]
        response = requests.get(logo_url)
        img = Image.open(BytesIO(response.content))

        xy = [bar.get_width() + 3, y]
        img = OffsetImage(img, zoom=0.05)
        img.image.axes = ax
        ab = AnnotationBbox(img, xy, xycoords='data', frameon=False, pad=0)
        ax.add_artist(ab)

    if top_or_bottom:
        # Add labels inside bars
        i = 1
        for bar, score in zip(bars, scores):
            ax.text(bar.get_width() - 3, bar.get_y() + bar.get_height() / 2, 
                    f"#{i} - {score:.1f}", va='center', ha='right', fontsize=8, color='white', fontweight='bold')
            i = i + 1
        ax.set_yticks(y_pos)
        ax.tick_params(axis='y', length=0)
        ax.set_yticklabels([])
        ax.set_yticks([])
        # Tidy up the axis
        ax.invert_yaxis() 
        ax.set_xlim([0, 105])  # Adjust as per your score range
        ax.set_xticks([])  # Remove x-ticks
        ax.set_xticklabels([])  # Remove x-labels
    else:
        i = 110
        for bar, score in zip(bars, scores):
            ax.text(bar.get_width() - bar.get_width() - 1, bar.get_y() + bar.get_height() / 2, 
                    f"#{i} - {score:.1f}", va='center', ha='right', fontsize=8, color='white', fontweight='bold')
            i = i + 1
        ax.set_yticks(y_pos)
        ax.tick_params(axis='y', length=0)
        ax.set_yticklabels([])
        ax.set_yticks([])
        # Tidy up the axis
        ax.invert_yaxis() 
        ax.set_xlim([0, 100])  # Adjust as per your score range
        ax.set_xticks([])  # Remove x-ticks
        ax.set_xticklabels([])  # Remove x-labels

    ax.set_title(title, fontsize=12, fontweight='bold', color='black')

    # Remove axes spines
    for spine in ['top', 'left', 'bottom', 'right']:
        ax.spines[spine].set_visible(False)

def other_best_and_worst(all_data, logos, metric, title, subtitle):
    if metric == 'avg_expected_wins':
        top_25_best = all_data.sort_values(metric, ascending=True)[:25].reset_index(drop=True)
        top_25_worst = all_data.sort_values(metric, ascending=False)[:25].reset_index(drop=True)
        top_25_worst = top_25_worst.sort_values(metric, ascending=True)[:25].reset_index(drop=True)
        rounding = 3
        good = 'Hardest'
        bad = 'Easiest'
    else:
        top_25_best = all_data.sort_values(metric, ascending=False)[:25].reset_index(drop=True)
        top_25_worst = all_data.sort_values(metric, ascending=True)[:25].reset_index(drop=True)
        top_25_worst = top_25_worst.sort_values(metric, ascending=False)[:25].reset_index(drop=True)
        rounding = 1
        if (metric == 'wins_above_good') or (metric == 'performance'):
            rounding = 3
        if (metric == 'RTP') or (metric == 'wins_above_average'):
            rounding = 2
        good = 'Best'
        bad = 'Worst'
        if metric == 'performance':
            good='Overperformers'
            bad='Underperformers'
        if (metric == 'wins_above_average'):
            good = 'Most'
            bad = 'Least'

    # Create a figure with 5 rows and 10 columns
    fig, axs = plt.subplots(5, 10, figsize=(20, 10), dpi=125)

    # Adjust space between subplots
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    fig.patch.set_facecolor('#CECEB2')

    # Title and description
    plt.suptitle(title, fontsize=20, fontweight='bold', color='black')
    plt.text(0.51, 0.935, subtitle, ha='center', fontsize=14, color='black', transform=fig.transFigure)
    plt.text(0.9, 0.09, "@PEARatings", ha='right', fontsize=18, color='black', transform=fig.transFigure, fontweight='bold')

    # Fill the grid alternating Best and Worst Defenses
    for i in range(5):  # There are 5 rows
        # Best defenses: Columns 0-4 for each row (Best in every odd index)
        for j in range(5):  
            ax = axs[i, j]
            team = top_25_best.loc[i*5 + j, 'team']
            logo_url = logos[logos['team'] == team]['logo'].values[0][0]
            response = requests.get(logo_url)
            img = Image.open(BytesIO(response.content))
            ax.imshow(img)
            ax.set_facecolor('#f0f0f0')
            ax.set_title(f"#{i*5 + j + 1} \n{round(top_25_best.loc[i*5 + j, metric], rounding)}", fontsize=14, fontweight='bold')
            ax.axis('off')

        # Worst defenses: Columns 5-9 for each row (Worst in every even index after 5)
        for j in range(5, 10):  
            ax = axs[i, j]
            team = top_25_worst.loc[i*5 + (j-5), 'team']
            logo_url = logos[logos['team'] == team]['logo'].values[0][0]
            response = requests.get(logo_url)
            img = Image.open(BytesIO(response.content))
            ax.imshow(img)
            ax.set_facecolor('#f0f0f0')
            
            # Start counting for Worst from 134 and decrement
            worst_rank = (len(all_data) - 24) + (i*5 + (j-5)) 
            ax.set_title(f"#{worst_rank} \n{round(top_25_worst.loc[i*5 + (j-5), metric], rounding)}", fontsize=14, fontweight='bold')
            ax.axis('off')

    fig.add_artist(Line2D([0.512, 0.512], [0.12, 0.92], color='black', lw=5))
    fig.text(0.13, 0.965, good, ha='left', va='center', fontsize=20, fontweight='bold', color='black')
    fig.text(0.89, 0.965, bad, ha='right', va='center', fontsize=20, fontweight='bold', color='black')

    # Show the final figure
    plt.show()

def simulate_game_known(home_team, away_team, home_win_prob):
    """Simulates a single game between two teams based on home win probability."""
    random_outcome = np.random.random() * 100  # Generates a number between 0 and 100
    if random_outcome < home_win_prob:
        return home_team, away_team  # Home team wins, Away team loses
    else:
        return away_team, home_team  # Away team wins, Home team loses

def simulate_season_known(schedules, team_data):
    """Simulates one season based on current power ratings and schedule."""
    team_wins = {team: 0 for team in team_data['team'].unique()}  # Initialize win counts
    team_losses = {team: 0 for team in team_data['team'].unique()}  # Initialize loss counts

    for _, game in schedules.iterrows():
        home_team = game['home_team']
        away_team = game['away_team']
        home_win_prob = game['PEAR_win_prob']

        # Simulate the game outcome
        winner, loser = simulate_game_known(home_team, away_team, home_win_prob)
        team_wins[winner] += 1  # Increment win count for the winner
        team_losses[loser] += 1  # Increment loss count for the loser

    return team_wins, team_losses  # Returns the win and loss counts for all teams at the end of the season

def monte_carlo_simulation_known(num_simulations, schedules, team_data):
    """Runs a Monte Carlo simulation over multiple seasons."""
    win_results = []
    loss_results = []

    for _ in range(num_simulations):
        season_wins, season_losses = simulate_season_known(schedules, team_data)
        win_results.append(season_wins)
        loss_results.append(season_losses)
    
    return win_results, loss_results

def analyze_simulation_known(win_results, loss_results, schedules, records):
    """Aggregates the results of multiple simulated seasons and calculates win-loss records."""
    # Convert list of results into DataFrames
    win_df = pd.DataFrame(win_results)
    loss_df = pd.DataFrame(loss_results)
    
    # Determine the number of games each team plays
    game_counts = schedules.groupby('home_team').size() + schedules.groupby('away_team').size()
    game_counts = game_counts.groupby(level=0).sum()  # Combine home and away counts

    # Add 1 win for teams with only 11 games
    for team in win_df.columns:
        win_df[team] += records[records['team'] == team]['wins'].values[0]
        loss_df[team] += records[records['team'] == team]['losses'].values[0]

    for team in win_df.columns:
        team_games_played = records[records['team'] == team]['games_played'].values[0]
        if (team_games_played == 12) | (team_games_played == 13) | (team == 'Liberty') | (team == 'App State'):
            continue
        else:
            if (team_games_played+game_counts[team]) == 11:
                win_df[team] += 1
            if (team_games_played+game_counts[team]) == 10:
                win_df[team] += 2
        
    # Calculate average win totals for each team
    avg_wins = win_df.mean(axis=0)
    
    # Calculate average loss totals for each team
    avg_losses = loss_df.mean(axis=0)
    
    # Calculate the mode (most frequent number of wins) for each team
    most_common_wins = win_df.mode(axis=0).iloc[0]  # mode() returns a DataFrame, take the first row
    
    # Calculate the mode (most frequent number of losses) for each team
    most_common_losses = loss_df.mode(axis=0).iloc[0]  # mode() returns a DataFrame, take the first row
    
    # Combine wins and losses into records
    most_common_records = pd.DataFrame({
        'Wins': most_common_wins,
        'Losses': most_common_losses
    })
    
    win_thresholds = {}
    for wins in range(13):  # 0 to 12 wins
        win_thresholds[f'win_{wins}'] = win_df.apply(lambda x: (x == wins).sum() / len(x), axis=0)

    win_thresholds['WIN6%'] = win_df.apply(lambda x: (x >= 6).sum() / len(x), axis=0)
    
    # Create the win threshold DataFrame
    win_thresholds_df = pd.DataFrame(win_thresholds)
    win_thresholds_df.insert(0, 'team', win_df.columns)
    win_thresholds_df = win_thresholds_df.reset_index(drop=True)
    win_thresholds_df['expected_wins'] = list(avg_wins)
    win_thresholds_df['expected_loss'] = list(avg_losses)
    win_thresholds_df['projected_wins'] = list(most_common_records['Wins'])
    win_thresholds_df['projected_losses'] = list(most_common_records['Losses'])

    return win_thresholds_df

def fetch_logo_image(logo_url):
    response = requests.get(logo_url)
    return Image.open(BytesIO(response.content))

def average_team_distribution(num_simulations, schedules, average, team_name):

    def simulate_game_average(win_prob):
        random_outcome = np.random.random() * 100  # Generates a number between 0 and 100
        if random_outcome < win_prob:
            return "W"  # Home team wins, Away team loses
        else:
            return "L"  # Away team wins, Home team loses
        
    def simulate_season_average(schedules, team_name, average):
        wins = 0
        losses = 0
        for _, game in schedules.iterrows():
            if game['home_team'] == team_name:
                opponent_team = game['away_team']
                opponent_pr = game['away_pr']
                win_prob = PEAR_Win_Prob(average, opponent_pr)

                # opponent_elo = game['away_elo']
                # win_prob = round((10**((average-opponent_elo) / 400)) / ((10**((average-opponent_elo) / 400)) + 1)*100, 2)
            else:
                opponent_team = game['home_team']
                opponent_pr = game['home_pr']
                win_prob = 100 - PEAR_Win_Prob(opponent_pr, average)

                # opponent_elo = game['home_elo']
                # win_prob = 100 - round((10**((opponent_elo-average) / 400)) / ((10**((opponent_elo-average) / 400)) + 1)*100, 2)
            
            outcome = simulate_game_average(win_prob)
            if outcome == "W":
                wins += 1
            else:
                losses += 1

        return wins, losses
        
    def monte_carlo_simulation_average(num_simulations, schedules, average, team_name):
        """Runs a Monte Carlo simulation for an average team over multiple seasons."""
        win_results = []
        loss_results = []

        for _ in range(num_simulations):
            wins, losses = simulate_season_average(schedules, team_name, average)
            win_results.append(wins)
            loss_results.append(losses)
        
        return win_results, loss_results

    import statistics
    from collections import Counter
    def analyze_simulation_average(win_results, loss_results, schedules):
        games_played = len(schedules)
        if games_played == 11:
            win_results = [x + .948 for x in win_results]
        elif games_played == 10:
            win_results = [x + (2 * .948) for x in win_results]
    
        avg_wins = statistics.mean(win_results)
        avg_loss = statistics.mean(loss_results)
        most_common_win = statistics.mode(win_results)
        most_common_loss = statistics.mode(loss_results)


        win_counts = Counter(win_results)    
        total_simulations = len(win_results)
        win_percentages = {f"win_{wins}": (win_counts[wins] / total_simulations) for wins in range(13)}
        win_thresholds = pd.DataFrame([win_percentages])
        
        # win_thresholds = {}
        # for wins in range(13):  # 0 to 12 wins
        #     win_thresholds[f'win_{wins}'] = win_df.apply(lambda x: (x == wins).sum() / len(x), axis=0)

        win_thresholds['WIN6%'] = win_thresholds.loc[:, 'win_6':'win_12'].sum(axis=1)
        win_thresholds['expected_wins'] = avg_wins
        win_thresholds['expected_losses'] = avg_loss
        win_thresholds['projected_wins'] = most_common_win
        win_thresholds['projected_losses'] = most_common_loss

        return win_thresholds
    
    avg_win, avg_loss = monte_carlo_simulation_average(num_simulations, schedules, average, team_name)
    win_thresholds = analyze_simulation_average(avg_win, avg_loss,schedules)
    return win_thresholds

def finish_schedule(win_thresholds_in_season, team_data, logos, team):
    # Prepare the figure
    fig, ax = plt.subplots(figsize=(10, 10))
    fig.patch.set_facecolor('#CECEB2')

    # Filter for the specific team and get the team logo
    wins = win_thresholds_in_season[win_thresholds_in_season['team'] == team]
    expected_wins = round(win_thresholds_in_season[win_thresholds_in_season['team'] == team]['expected_wins'].values[0], 1)
    expected_losses = round(win_thresholds_in_season[win_thresholds_in_season['team'] == team]['expected_loss'].values[0], 1)
    rank = team_data[team_data['team'] == team]['Unnamed: 0'].values[0]
    team_url = logos[logos['team'] == team]['logo'].values[0][0]
    team_logo = fetch_logo_image(team_url)

    # Adjusted extent to center the logo
    ax.imshow(team_logo, extent=(0.45, 0.5, 0.85, 0.9))

    # Iterate over the 'win_0' to 'win_12' columns and display their values
    y_pos = 0.805  # Start the text higher up
    for i in range(13):
        if i==7:
            y_pos=0.81
        prob_win = 100 * wins[f'win_{i}'].values[0]
        if prob_win == 0:
            prob_win = None
            win_text = f"{i} Wins: None"
        else:
            win_text = f"{i} Wins: {prob_win:.2f}%"
        if i<7:
            ax.text(0.458, y_pos, win_text, fontsize=13, color='black', horizontalalignment='center')
        else:
            ax.text(0.492, y_pos, win_text, fontsize=13, color='black', horizontalalignment='center')
        y_pos += 0.005

    # Display the team name at the center
    ax.text(0.475, 0.845, f"#{rank} {team}", fontsize=14, fontweight='bold', color='black', horizontalalignment='center')
    ax.text(0.475, 0.84, f"Expected Record: {expected_wins} - {expected_losses}", color='black', horizontalalignment='center')

    # Remove axes for a cleaner look
    ax.axis('off')

    # Center the plot title
    plt.title("Simulated Wins", fontsize=12, fontweight='bold', verticalalignment='center')

    # Ensure the layout fits well
    plt.tight_layout()

    # Show the plot
    plt.show()

def simulate_game_conference(home_team, away_team, home_win_prob):
    """Simulates a single game between two teams based on home win probability."""
    random_outcome = np.random.random() * 100  # Generates a number between 0 and 100
    if random_outcome < home_win_prob:
        return home_team, away_team  # Home team wins, Away team loses
    else:
        return away_team, home_team  # Away team wins, Home team loses

def simulate_season_conference(schedules, team_data):
    """Simulates one season based on current power ratings and schedule."""
    team_wins = {team: 0 for team in team_data['team'].unique()}  # Initialize win counts
    team_losses = {team: 0 for team in team_data['team'].unique()}  # Initialize loss counts

    for _, game in schedules.iterrows():
        home_team = game['home_team']
        away_team = game['away_team']
        home_win_prob = game['PEAR_win_prob']

        # Simulate the game outcome
        winner, loser = simulate_game_conference(home_team, away_team, home_win_prob)
        team_wins[winner] += 1  # Increment win count for the winner
        team_losses[loser] += 1  # Increment loss count for the loser

    return team_wins, team_losses  # Returns the win and loss counts for all teams at the end of the season

def monte_carlo_simulation_conference(num_simulations, schedules, team_data):
    """Runs a Monte Carlo simulation over multiple seasons."""
    win_results = []
    loss_results = []

    for _ in range(num_simulations):
        season_wins, season_losses = simulate_season_conference(schedules, team_data)
        win_results.append(season_wins)
        loss_results.append(season_losses)
    
    return win_results, loss_results

def analyze_simulation_conference(win_results, loss_results, schedules, records):
    """Aggregates the results of multiple simulated seasons and calculates win-loss records."""
    # Convert list of results into DataFrames
    win_df = pd.DataFrame(win_results)
    loss_df = pd.DataFrame(loss_results)
    
    # Determine the number of games each team plays
    game_counts = schedules.groupby('home_team').size() + schedules.groupby('away_team').size()
    game_counts = game_counts.groupby(level=0).sum()  # Combine home and away counts

    # Add 1 win for teams with only 11 games
    for team in win_df.columns:
        win_df[team] += records[records['team'] == team]['conference_wins'].values[0]
        loss_df[team] += records[records['team'] == team]['conference_losses'].values[0]

    # for team in win_df.columns:
    #     team_games_played = records[records['team'] == team]['games_played'].values[0]
    #     if (team_games_played+game_counts[team]) == 11:
    #         win_df[team] += 1
    #     if (team_games_played+game_counts[team]) == 10:
    #         win_df[team] += 2
        
    # Calculate average win totals for each team
    avg_wins = win_df.mean(axis=0)
    
    # Calculate average loss totals for each team
    avg_losses = loss_df.mean(axis=0)
    
    # Calculate the mode (most frequent number of wins) for each team
    most_common_wins = win_df.mode(axis=0).iloc[0]  # mode() returns a DataFrame, take the first row
    
    # Calculate the mode (most frequent number of losses) for each team
    most_common_losses = loss_df.mode(axis=0).iloc[0]  # mode() returns a DataFrame, take the first row
    
    # Combine wins and losses into records
    most_common_records = pd.DataFrame({
        'Wins': most_common_wins,
        'Losses': most_common_losses
    })
    
    win_thresholds = {}
    for wins in range(10):  # 0 to 12 wins
        win_thresholds[f'win_{wins}'] = win_df.apply(lambda x: (x == wins).sum() / len(x), axis=0)
    
    # Create the win threshold DataFrame
    win_thresholds_df = pd.DataFrame(win_thresholds)
    win_thresholds_df.insert(0, 'team', win_df.columns)
    win_thresholds_df = win_thresholds_df.reset_index(drop=True)
    win_thresholds_df['expected_wins'] = list(avg_wins)
    win_thresholds_df['expected_loss'] = list(avg_losses)
    win_thresholds_df['projected_wins'] = list(most_common_records['Wins'])
    win_thresholds_df['projected_losses'] = list(most_common_records['Losses'])

    return win_thresholds_df

def plot_matchup(wins_df, all_conference_wins, logos_df, team_data, last_week_data, last_month_data, start_season_data, all_data, schedule_info, records, SOS, SOR, most_deserving, home_team, away_team, neutrality=False):
    sns.set(style='whitegrid')
    ################################# HELPER FUNCTIONS #################################

    def PEAR_Win_Prob(home_power_rating, away_power_rating):
        return round((1 / (1 + 10 ** ((away_power_rating - (home_power_rating)) / 20.5))) * 100, 2)

    def adjust_home_pr(home_win_prob):
        return ((home_win_prob - 50) / 50) * 5

    def round_to_nearest_half(x):
        return np.round(x * 2) / 2   

    def find_team_spread(game, team_data, team_name, home_team, away_team, neutral):
        home_rating = team_data[team_data['team'] == home_team]['power_rating'].values[0]
        away_rating = team_data[team_data['team'] == away_team]['power_rating'].values[0]
        home_win_prob = game['home_win_prob']
        def adjust_home_pr(home_win_prob):
            return ((home_win_prob - 50) / 50) * 5
        def round_to_nearest_half(x):
            return np.round(x * 2) / 2 
        if neutral:
            spread = round((home_rating + adjust_home_pr(home_win_prob) - away_rating),1)
        else:
            spread = round((4.6 + home_rating + adjust_home_pr(home_win_prob) - away_rating),1)
        if (home_team == team_name) & (spread > 0):
            output = "-" + str(spread)
        elif (home_team == team_name) & (spread < 0):
            output = "+" + str(abs(spread))
        elif (home_team != team_name) & (spread < 0):
            output = str(spread)
        elif (spread == 0.0):
            output = str(spread)
        else:
            output = "+" + str(spread)

        return output, spread
    
    def add_spreads(schedule_info, team_data, team_name):
        # Define a helper function to apply the find_team_spread to each row
        def compute_spread(row):
            # Extract relevant data from the current row
            home_team = row['home_team']
            away_team = row['away_team']
            neutral = row['neutral']
            
            # Call the find_team_spread function with this row's data
            spread_str, raw_spread = find_team_spread(row, team_data, team_name, home_team, away_team, neutral)
            
            # Return both values as a tuple, so we can unpack them into two columns
            return pd.Series([spread_str, raw_spread])

        # Apply the compute_spread function to each row of the DataFrame
        schedule_info[['spread', 'raw_spread']] = schedule_info.apply(compute_spread, axis=1)
        
        return schedule_info

    def get_color_wl(w_l):
        if w_l == "W":
            return '#1D4D00'
        else:
            return '#660000'
    
    def get_color_future(raw_spread, game_home_team, team):
        if (game_home_team == team) and (raw_spread <= 0):
            return '#660000'
        if (game_home_team == team) and (raw_spread > 0):
            return '#1D4D00'
        if (game_home_team != team) and (raw_spread <= 0):
            return '#1D4D00'
        if (game_home_team != team) and (raw_spread > 0):
            return '#660000'
        
    def get_rank_color(number):
        """
        Returns a hex color code based on a number between 1 and 134.
        
        Parameters:
            number (int): A number between 1 and 134.
        
        Returns:
            str: Hex color code corresponding to the input number.
        """
        if not 1 <= number <= 134:
            raise ValueError("Number must be between 1 and 134")
        
        # Define the color gradient points
        gradient = [
            (1, '#1D4D00'),    # Start green
            (35, '#2C5E00'),   # Midpoint green
            (67, '#808080'),   # Grey
            (105, '#8B0000'),  # Red in the middle
            (134, '#660000')   # End dark red
        ]
        
        # Convert hex to RGB
        def hex_to_rgb(hex_color):
            hex_color = hex_color.lstrip('#')
            return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
        
        # Convert RGB to hex
        def rgb_to_hex(rgb):
            return '#' + ''.join(f'{int(c):02X}' for c in rgb)
        
        # Interpolate between two colors
        def interpolate_color(color1, color2, fraction):
            return tuple(
                color1[i] + (color2[i] - color1[i]) * fraction
                for i in range(3)
            )
        
        # Find the range that includes the number
        for i in range(len(gradient) - 1):
            start, end = gradient[i], gradient[i + 1]
            if start[0] <= number <= end[0]:
                fraction = (number - start[0]) / (end[0] - start[0])
                start_rgb, end_rgb = hex_to_rgb(start[1]), hex_to_rgb(end[1])
                interpolated_rgb = interpolate_color(start_rgb, end_rgb, fraction)
                return rgb_to_hex(interpolated_rgb)
        
        # Fallback (should not reach here)
        return '#000000'

    def grab_team_elo(team):
        if postseason == True:
            elo_ratings_list = [*ratings_api.get_elo_ratings(year=current_year, team=team)]
            elo_ratings_dict = [dict(
                team=e.team,
                elo=e.elo
            ) for e in elo_ratings_list]
            elo_ratings = pd.DataFrame(elo_ratings_dict)
        else:
            elo_ratings_list = [*ratings_api.get_elo_ratings(year=current_year, week=current_week, team=team)]
            elo_ratings_dict = [dict(
                team=e.team,
                elo=e.elo
            ) for e in elo_ratings_list]
            elo_ratings = pd.DataFrame(elo_ratings_dict)        
        return elo_ratings['elo'].values[0]
    
    ################################# PREPPING DATA NEEDED #################################

    entire_schedule = schedule_info.copy()
    completed_games = schedule_info[schedule_info['home_points'].notna()]
    non_completed_games = schedule_info[schedule_info['home_points'].isna()]
    scaler100 = MinMaxScaler(feature_range=(1,100))
    all_data['talent_scaled_percentile'] = scaler100.fit_transform(all_data[['talent_scaled']])

    home_wins_df = wins_df[wins_df['team'] == home_team]
    away_wins_df = wins_df[wins_df['team'] == away_team]
    # home_xwins = round(wins_df[wins_df['team'] == home_team]['expected_wins'].values[0], 1)
    # away_xwins = round(wins_df[wins_df['team'] == away_team]['expected_wins'].values[0], 1)
    # home_xlosses = round(wins_df[wins_df['team'] == home_team]['expected_loss'].values[0], 1)
    # away_xlosses = round(wins_df[wins_df['team'] == away_team]['expected_loss'].values[0], 1)
    home_rank = team_data[team_data['team'] == home_team].index[0] + 1
    away_rank = team_data[team_data['team'] == away_team].index[0] + 1
    home_win_6 = round(100 * home_wins_df['WIN6%'].values[0], 1)
    away_win_6 = round(100 * away_wins_df['WIN6%'].values[0], 1)
    home_last_week = last_week_data[last_week_data['team'] == home_team]['power_rating'].values[0]
    away_last_week = last_week_data[last_week_data['team'] == away_team]['power_rating'].values[0]
    home_last_month = last_month_data[last_month_data['team'] == home_team]['power_rating'].values[0]
    away_last_month = last_month_data[last_month_data['team'] == away_team]['power_rating'].values[0]
    home_start_rating = start_season_data[start_season_data['team'] == home_team]['power_rating'].values[0]
    away_start_rating = start_season_data[start_season_data['team'] == away_team]['power_rating'].values[0]
    home_wins = records[records['team'] == home_team]['wins'].values[0]
    home_losses = records[records['team'] == home_team]['losses'].values[0]
    home_conference_wins = records[records['team'] == home_team]['conference_wins'].values[0]
    home_conference_losses = records[records['team'] == home_team]['conference_losses'].values[0]
    away_wins = records[records['team'] == away_team]['wins'].values[0]
    away_losses = records[records['team'] == away_team]['losses'].values[0]
    away_conference_wins = records[records['team'] == away_team]['conference_wins'].values[0]
    away_conference_losses = records[records['team'] == away_team]['conference_losses'].values[0]

    average_pr = team_data['power_rating'].mean()
    home_completed_games = completed_games[(completed_games['home_team'] == home_team) | (completed_games['away_team'] == home_team)]
    away_completed_games = completed_games[(completed_games['home_team'] == away_team) | (completed_games['away_team'] == away_team)]
    home_completed_games['team_win_prob'] = np.where(home_completed_games['home_team'] == home_team, 
                                    home_completed_games['PEAR_win_prob'], 
                                    100 - home_completed_games['PEAR_win_prob'])
    away_completed_games['team_win_prob'] = np.where(away_completed_games['home_team'] == away_team, 
                                    away_completed_games['PEAR_win_prob'], 
                                    100 - away_completed_games['PEAR_win_prob'])
    home_completed_games['avg_win_prob'] = np.where(home_completed_games['home_team'] == home_team, 
                                    PEAR_Win_Prob(average_pr, home_completed_games['away_pr']), 
                                    100 - PEAR_Win_Prob(home_completed_games['home_pr'], average_pr))
    away_completed_games['avg_win_prob'] = np.where(away_completed_games['home_team'] == away_team, 
                                    PEAR_Win_Prob(average_pr, away_completed_games['away_pr']), 
                                    100 - PEAR_Win_Prob(away_completed_games['home_pr'], average_pr))
    home_avg_xwins = round(sum(home_completed_games['avg_win_prob']) / 100, 1)
    home_avg_xlosses = round(len(home_completed_games) - home_avg_xwins, 1)
    away_avg_xwins = round(sum(away_completed_games['avg_win_prob']) / 100, 1)
    away_avg_xlosses = round(len(away_completed_games) - away_avg_xwins, 1)
    home_completed_xwins = round(sum(home_completed_games['team_win_prob']) / 100, 1)
    away_completed_xwins = round(sum(away_completed_games['team_win_prob']) / 100, 1)
    home_completed_xlosses = round(len(home_completed_games) - home_completed_xwins, 1)
    away_completed_xlosses = round(len(away_completed_games) - away_completed_xwins, 1)
    home_games_played = home_wins + home_losses
    away_games_played = away_wins + away_losses
    if len(home_completed_games) != home_games_played:
        home_completed_xwins = home_completed_xwins + 1
        home_avg_xwins = home_avg_xwins + 1
    if len(away_completed_games) != away_games_played:
        away_completed_xwins = away_completed_xwins + 1
        away_avg_xwins = away_avg_xwins + 1
    home_win_out = home_wins + 12 - home_games_played
    away_win_out = away_wins + 12 - away_games_played
    home_win_out_percent = round(home_wins_df[f'win_{home_win_out}'].values[0] * 100, 1)
    away_win_out_percent = round(away_wins_df[f'win_{away_win_out}'].values[0] * 100, 1)
    home_md = most_deserving[most_deserving['team'] == home_team]['Performance'].values[0]
    away_md = most_deserving[most_deserving['team'] == away_team]['Performance'].values[0]
    home_sor = SOR[SOR['team'] == home_team]['SOR'].values[0]
    home_sos = SOS[SOS['team'] == home_team]['SOS'].values[0]
    away_sor = SOR[SOR['team'] == away_team]['SOR'].values[0]
    away_sos = SOS[SOS['team'] == away_team]['SOS'].values[0]
    home_non_completed_games = non_completed_games[(non_completed_games['home_team'] == home_team) | (non_completed_games['away_team'] == home_team)].reset_index(drop = True)
    away_non_completed_games = non_completed_games[(non_completed_games['home_team'] == away_team) | (non_completed_games['away_team'] == away_team)].reset_index(drop = True)
    home_elo = grab_team_elo(home_team)
    away_elo = grab_team_elo(away_team)

    cmap = LinearSegmentedColormap.from_list('dark_gradient_orange', ['#660000', '#8B0000', '#808080', '#2C5E00', '#1D4D00'], N=100)
    def get_color(value, vmin=0, vmax=100):
        norm_value = (value - vmin) / (vmax - vmin)  # Normalize the value between 0 and 1
        return cmap(norm_value)  # Get the color from the colormap
    
    xwin = LinearSegmentedColormap.from_list('dark_gradient_pattern', ['#660000', '#8B0000', '#808080', '#2C5E00', '#1D4D00'], N=100)

    # Function to get color based on value with specified boundaries
    def get_gradient_color(value, vmin=-1.5, vmax=1.5):
        if value <= vmin:
            return '#660000'  # Lighter dark red for <= -1.5
        elif value >= vmax:
            return '#1D4D00'  # Dark green for >= 1.5
        else:
            norm_value = (value - vmin) / (vmax - vmin)  # Normalize value between vmin and vmax
            return xwin(norm_value)  # Get color from colormap for intermediate values

    if (len(home_non_completed_games) != 0):
        home_non_completed_games = add_spreads(home_non_completed_games, team_data, home_team)
    if (len(away_non_completed_games) != 0):
        away_non_completed_games = add_spreads(away_non_completed_games, team_data, away_team)

    home_power_rating = round(team_data[team_data['team'] == home_team]['power_rating'].values[0], 2)
    home_talent_scaled = round(all_data[all_data['team'] == home_team]['talent_scaled_percentile'].values[0], 2)
    home_offense_success = round(all_data[all_data['team'] == home_team]['offense_success_scaled'].values[0], 2)
    home_defense_success = round(all_data[all_data['team'] == home_team]['defense_success_scaled'].values[0], 2)
    home_offense_explosive = round(all_data[all_data['team'] == home_team]['offense_explosive'].values[0], 2)
    home_defense_explosive = round(all_data[all_data['team'] == home_team]['defense_explosive'].values[0], 2)
    home_turnovers = round(all_data[all_data['team'] == home_team]['total_turnovers_scaled'].values[0], 2)
    home_penalties = round(all_data[all_data['team'] == home_team]['penalties_scaled'].values[0], 2)
    home_offensive = all_data[all_data['team'] == home_team]['offensive_rank'].values[0]
    home_defensive = all_data[all_data['team'] == home_team]['defensive_rank'].values[0]
    home_stm = int(all_data[all_data['team'] == home_team]['STM_rank'].values[0])
    home_pbr = int(all_data[all_data['team'] == home_team]['PBR_rank'].values[0])
    home_dce = int(all_data[all_data['team'] == home_team]['DCE_rank'].values[0])
    home_dde = int(all_data[all_data['team'] == home_team]['DDE_rank'].values[0])
    home_talent_rank = int(all_data[all_data['team'] == home_team]['talent_scaled_rank'].values[0])
    home_offense_success_rank = int(all_data[all_data['team'] == home_team]['offense_success_rank'].values[0])
    home_defense_success_rank = int(all_data[all_data['team'] == home_team]['defense_success_rank'].values[0])
    home_offense_explosive_rank = int(all_data[all_data['team'] == home_team]['offense_explosive_rank'].values[0])
    home_defense_explosive_rank = int(all_data[all_data['team'] == home_team]['defense_explosive_rank'].values[0])
    home_turnover_rank = int(all_data[all_data['team'] == home_team]['total_turnovers_rank'].values[0])
    home_penalties_rank = int(all_data[all_data['team'] == home_team]['penalties_rank'].values[0])

    away_power_rating = round(team_data[team_data['team'] == away_team]['power_rating'].values[0], 2)
    away_talent_scaled = round(all_data[all_data['team'] == away_team]['talent_scaled_percentile'].values[0], 2)
    away_offense_success = round(all_data[all_data['team'] == away_team]['offense_success_scaled'].values[0], 2)
    away_defense_success = round(all_data[all_data['team'] == away_team]['defense_success_scaled'].values[0], 2)
    away_offense_explosive = round(all_data[all_data['team'] == away_team]['offense_explosive'].values[0], 2)
    away_defense_explosive = round(all_data[all_data['team'] == away_team]['defense_explosive'].values[0], 2)
    away_turnovers = round(all_data[all_data['team'] == away_team]['total_turnovers_scaled'].values[0], 2)
    away_penalties = round(all_data[all_data['team'] == away_team]['penalties_scaled'].values[0], 2)
    away_offensive = all_data[all_data['team'] == away_team]['offensive_rank'].values[0]
    away_defensive = all_data[all_data['team'] == away_team]['defensive_rank'].values[0]
    away_stm = int(all_data[all_data['team'] == away_team]['STM_rank'].values[0])
    away_pbr = int(all_data[all_data['team'] == away_team]['PBR_rank'].values[0])
    away_dce = int(all_data[all_data['team'] == away_team]['DCE_rank'].values[0])
    away_dde = int(all_data[all_data['team'] == away_team]['DDE_rank'].values[0])
    away_talent_rank = int(all_data[all_data['team'] == away_team]['talent_scaled_rank'].values[0])
    away_offense_success_rank = int(all_data[all_data['team'] == away_team]['offense_success_rank'].values[0])
    away_defense_success_rank = int(all_data[all_data['team'] == away_team]['defense_success_rank'].values[0])
    away_offense_explosive_rank = int(all_data[all_data['team'] == away_team]['offense_explosive_rank'].values[0])
    away_defense_explosive_rank = int(all_data[all_data['team'] == away_team]['defense_explosive_rank'].values[0])
    away_turnover_rank = int(all_data[all_data['team'] == away_team]['total_turnovers_rank'].values[0])
    away_penalties_rank = int(all_data[all_data['team'] == away_team]['penalties_rank'].values[0])

    home_win_prob = round((10**((home_elo - away_elo) / 400)) / ((10**((home_elo - away_elo) / 400)) + 1)*100,2)
    PEAR_home_prob = PEAR_Win_Prob(home_power_rating, away_power_rating)
    spread = (4.6 + home_power_rating + adjust_home_pr(home_win_prob) - away_power_rating).round(1)
    if neutrality:
        spread = (spread - 4.6).round(1)
    spread = round(spread,1)
    if (spread) <= 0:
        formatted_spread = (f'{away_team} {spread}')
        game_win_prob = round(100 - PEAR_home_prob,2)
    elif (spread) > 0:
        formatted_spread = (f'{home_team} -{spread}')
        game_win_prob = PEAR_home_prob

    ################################# PLOTTING LOGOS #################################

    fig, ax = plt.subplots(nrows=1, figsize=(12, 10),dpi=125)
    fig_width, fig_height = fig.get_size_inches()
    fig.patch.set_facecolor('#CECEB2')  # Set figure background color
    ax.set_facecolor('#CECEB2') 
    ax.axis('off')
    home_logo_url = logos_df[logos_df['team'] == home_team]['logo'].values[0][0]
    home_team_color = logos_df[logos_df['team'] == home_team]['color'].values[0]   
    away_logo_url = logos_df[logos_df['team'] == away_team]['logo'].values[0][0]
    away_team_color = logos_df[logos_df['team'] == away_team]['color'].values[0]   
    
    response = requests.get(home_logo_url)
    logo_img = Image.open(BytesIO(response.content))
    img_ax = fig.add_axes([-.1,-.05,0.4,0.4])
    img_ax.imshow(logo_img)
    img_ax.axis('off')

    response = requests.get(away_logo_url)
    logo_img = Image.open(BytesIO(response.content))
    img_ax = fig.add_axes([1.,-.05,0.4,0.4])
    img_ax.imshow(logo_img)
    img_ax.axis('off')

    ################################# SCHEDULE INFO ################################# 

    home_j = 0.29
    plt.text(0.65, .32, "FCS Games Not Included", fontsize = 16, va='top', ha='center', transform = ax.transAxes, fontweight='bold')
    for i, game in home_completed_games.iterrows():
        neutral = game['neutral']
        plt.text(0.295, home_j, f"{game['week']}", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
        if game['home_team'] == home_team:
            if game['home_points'] > game['away_points']:
                w_L = 'W'
            else:
                w_L = 'L'
        else:
            if game['home_points'] > game['away_points']:
                w_L = 'L'
            else:
                w_L = 'W'
        if neutral:
            if game['home_team'] == home_team:
                home_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                plt.text(0.325, home_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
                plt.text(0.355, home_j, f"{game['away_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
            else:
                home_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                plt.text(0.325, home_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
                plt.text(0.355, home_j, f"{game['home_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        elif game['home_team'] == home_team:
            home_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
            plt.text(0.325, home_j, f"vs", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
            plt.text(0.355, home_j, f"{game['away_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        else:
            home_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
            plt.text(0.325, home_j, f"@", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
            plt.text(0.355, home_j, f"{game['home_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        if game['home_points'] > game['away_points']:
            plt.text(0.61, home_j, f"{int(game['home_points'])}-{int(game['away_points'])}", fontsize = 16, va='top', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        else:
            plt.text(0.61, home_j, f"{int(game['away_points'])}-{int(game['home_points'])}", fontsize = 16, va='top', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        home_j -= 0.03
    if len(home_non_completed_games != 0):
        for i, game in home_non_completed_games.iterrows():
            game_home_team = game['home_team']
            neutral = game['neutral']
            plt.text(0.295, home_j, f"{game['week']}", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            spread = game['spread']
            raw_spread = game['raw_spread']
            if neutral:
                if game['home_team'] == home_team:
                    home_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                    plt.text(0.325, home_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                    plt.text(0.355, home_j, f"{game['away_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                else:
                    home_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                    plt.text(0.325, home_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                    plt.text(0.355, home_j, f"{game['home_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            elif game['home_team'] == home_team:
                home_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                plt.text(0.325, home_j, f"vs", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                plt.text(0.355, home_j, f"{game['away_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            else:
                home_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                plt.text(0.325, home_j, f"@", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                plt.text(0.355, home_j, f"{game['home_team']} (#{home_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            plt.text(0.61, home_j, f"{spread}", fontsize=16, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_color_future(raw_spread, game_home_team, home_team))

            home_j -= 0.03

    away_j = 0.29
    for i, game in away_completed_games.iterrows():
        neutral = game['neutral']
        plt.text(0.675, away_j, f"{game['week']}", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
        if game['home_team'] == away_team:
            if game['home_points'] > game['away_points']:
                w_L = 'W'
            else:
                w_L = 'L'
        else:
            if game['home_points'] > game['away_points']:
                w_L = 'L'
            else:
                w_L = 'W'
        if neutral:
            if game['home_team'] == away_team:
                away_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                plt.text(0.705, away_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
                plt.text(0.735, away_j, f"{game['away_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
            else:
                away_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                plt.text(0.705, away_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
                plt.text(0.735, away_j, f"{game['home_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        elif game['home_team'] == away_team:
            away_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
            plt.text(0.705, away_j, f"vs", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
            plt.text(0.735, away_j, f"{game['away_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        else:
            away_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
            plt.text(0.705, away_j, f"@", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
            plt.text(0.735, away_j, f"{game['home_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        if game['home_points'] > game['away_points']:
            plt.text(0.99, away_j, f"{int(game['home_points'])}-{int(game['away_points'])}", fontsize = 16, va='top', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        else:
            plt.text(0.99, away_j, f"{int(game['away_points'])}-{int(game['home_points'])}", fontsize = 16, va='top', transform=ax.transAxes, color = get_color_wl(w_L), fontweight='bold')
        away_j -= 0.03

    if (len(away_non_completed_games) != 0):
        for i, game in away_non_completed_games.iterrows():
            neutral = game['neutral']
            game_home_team = game['home_team']
            plt.text(0.675, away_j, f"{game['week']}", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            spread = game['spread']
            raw_spread = game['raw_spread']
            if neutral:
                if game['home_team'] == away_team:
                    away_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                    plt.text(0.705, away_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                    plt.text(0.735, away_j, f"{game['away_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                else:
                    away_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                    plt.text(0.705, away_j, f"(N)", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                    plt.text(0.735, away_j, f"{game['home_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            elif game['home_team'] == away_team:
                away_opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                plt.text(0.705, away_j, f"vs", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                plt.text(0.735, away_j, f"{game['away_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            else:
                away_opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                plt.text(0.705, away_j, f"@", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
                plt.text(0.735, away_j, f"{game['home_team']} (#{away_opponent_rank})", fontsize = 16, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
            plt.text(0.99, away_j, f"{spread}", fontsize=16, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_color_future(raw_spread, game_home_team, away_team))

            away_j -= 0.03

    plt.text(-0.075, 0.99, f"Power Rating: {home_power_rating} (#{home_rank})", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_rank))
    plt.text(-0.075, 0.94, f"Current Record: {home_wins} - {home_losses} ({home_conference_wins} - {home_conference_losses})", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold')
    plt.text(-0.075, 0.89, f"Current xRecord: {home_completed_xwins} - {home_completed_xlosses}", fontsize = 25, va='top', ha='left', transform=ax.transAxes)
    # plt.text(-0.075, 0.84, f"Win 6%: {home_win_6}%", fontsize = 25, va='top', ha='left', transform=ax.transAxes)
    # plt.text(-0.075, 0.79, f"Win Out%: {home_win_out_percent}%", fontsize = 25, va='top', ha='left', transform=ax.transAxes)
    plt.text(-0.075, 0.79, "Percentile (Rank)", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, fontweight='bold')
    plt.text(-0.075, 0.74, f"Team Talent: {home_talent_scaled} (#{home_talent_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_talent_scaled), fontweight='bold')
    plt.text(-0.075, 0.69, f"Offense Success: {home_offense_success} (#{home_offense_success_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_offense_success), fontweight='bold')
    plt.text(-0.075, 0.64, f"Offense Explosiveness: {home_offense_explosive} (#{home_offense_explosive_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_offense_explosive), fontweight='bold')
    plt.text(-0.075, 0.59, f"Defense Success: {home_defense_success} (#{home_defense_success_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_defense_success), fontweight='bold')
    plt.text(-0.075, 0.54, f"Defense Explosiveness: {home_defense_explosive} (#{home_defense_explosive_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_defense_explosive), fontweight='bold')
    plt.text(-0.075, 0.49, f"Turnovers: {home_turnovers} (#{home_turnover_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_turnovers), fontweight='bold')
    plt.text(-0.075, 0.44, f"Penalties: {home_penalties} (#{home_penalties_rank})", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, color=get_color(home_penalties), fontweight='bold')
    plt.text(-0.075, 0.39, f"SOS: #{home_sos}", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_sos))
    plt.text(0.15, 0.39, f"SOR: #{home_sor}", fontsize=25, verticalalignment='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_sor))

    plt.text(0.45, 0.69, f"MD: #{home_md}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_md))
    plt.text(0.45, 0.64, f"OFF: #{home_offensive}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_offensive))
    plt.text(0.45, 0.59, f"DEF: #{home_defensive}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_defensive))
    plt.text(0.45, 0.54, f"ST: #{home_stm}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_stm))
    plt.text(0.45, 0.49, f"PBR: #{home_pbr}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_pbr))
    plt.text(0.45, 0.44, f"DCE: #{home_dce}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_dce))
    plt.text(0.45, 0.39, f"DDE: #{home_dde}", fontsize = 25, va='top', ha='left', transform=ax.transAxes, fontweight='bold', color=get_rank_color(home_dde))


    plt.text(1.38, 0.99, f"Power Rating: {away_power_rating} (#{away_rank})", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_rank))
    plt.text(1.38, 0.94, f"Current Record: {away_wins} - {away_losses} ({away_conference_wins} - {away_conference_losses})", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold')
    plt.text(1.38, 0.89, f"Current xRecord: {away_completed_xwins} - {away_completed_xlosses}", fontsize = 25, va='top', ha='right', transform=ax.transAxes)
    # plt.text(1.38, 0.84, f"Win 6%: {away_win_6}%", fontsize = 25, va='top', ha='right', transform=ax.transAxes)
    # plt.text(1.38, 0.79, f"Win Out%: {away_win_out_percent}%", fontsize = 25, va='top', ha='right', transform=ax.transAxes)
    plt.text(1.38, 0.79, "Percentile (Rank)", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, fontweight='bold')
    plt.text(1.38, 0.74, f"Team Talent: {away_talent_scaled} (#{away_talent_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_talent_scaled), fontweight='bold')
    plt.text(1.38, 0.69, f"Offense Success: {away_offense_success} (#{away_offense_success_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_offense_success), fontweight='bold')
    plt.text(1.38, 0.64, f"Offense Explosiveness: {away_offense_explosive} (#{away_offense_explosive_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_offense_explosive), fontweight='bold')
    plt.text(1.38, 0.59, f"Defense Success: {away_defense_success} (#{away_defense_success_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_defense_success), fontweight='bold')
    plt.text(1.38, 0.54, f"Defense Explosiveness: {away_defense_explosive} (#{away_defense_explosive_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_defense_explosive), fontweight='bold')
    plt.text(1.38, 0.49, f"Turnovers: {away_turnovers} (#{away_turnover_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_turnovers), fontweight='bold')
    plt.text(1.38, 0.44, f"Penalties: {away_penalties} (#{away_penalties_rank})", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, color=get_color(away_penalties), fontweight='bold')
    plt.text(1.16, 0.39, f"SOS: #{away_sos}", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_sos))
    plt.text(1.38, 0.39, f"SOR: #{away_sor}", fontsize=25, verticalalignment='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_sor))

    plt.text(0.85, 0.69, f"MD: #{away_md}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_md))
    plt.text(0.85, 0.64, f"OFF: #{away_offensive}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_offensive))
    plt.text(0.85, 0.59, f"DEF: #{away_defensive}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_defensive))
    plt.text(0.85, 0.54, f"ST: #{away_stm}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_stm))
    plt.text(0.85, 0.49, f"PBR: #{away_pbr}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_pbr))
    plt.text(0.85, 0.44, f"DCE: #{away_dce}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_dce))
    plt.text(0.85, 0.39, f"DDE: #{away_dde}", fontsize = 25, va='top', ha='right', transform=ax.transAxes, fontweight='bold', color=get_rank_color(away_dde))

    plt.text(0.65, 0.965, f"PEAR Game Preview", fontsize = 45, va='top', ha='center', transform=ax.transAxes, fontweight='bold')
    plt.text(0.65, 0.895, f"@PEARatings", fontsize=25, va='top', ha='center', transform=ax.transAxes)
    plt.text(0.65, 0.85, f"{home_team} vs. {away_team}", fontsize = 35, va='top', ha='center', transform=ax.transAxes, fontweight='bold')
    plt.text(0.65, 0.79, f"{formatted_spread}", fontsize = 25, va='top', ha='center', transform=ax.transAxes, fontweight='bold')
    plt.text(0.65, 0.74, f"Win Prob: {game_win_prob}%", fontsize = 25, va='top', ha='center', transform=ax.transAxes, fontweight='bold')
    plt.text(0.65, 0.35, "Team Stats are Percentile Based", fontsize = 16, va='top', ha='center', transform=ax.transAxes, fontweight='bold')

    plt.tight_layout()
    plt.show()

def plot_win_probabilities(wins_df, all_conference_wins, logos_df, team_data, last_week_data, last_month_data, start_season_data, all_data, schedule_info, records, SOS, SOR, team_name, branding=True):
    # Set plot style
    sns.set(style="whitegrid")

    ################################# PREPPING DATA NEEDED #################################
    entire_schedule = schedule_info.copy()
    completed_games = schedule_info[schedule_info['home_points'].notna()]
    schedule_info = schedule_info[schedule_info['home_points'].isna()]

    wins_df = wins_df[wins_df['team'] == team_name]
    expected_wins = round(wins_df[wins_df['team'] == team_name]['expected_wins'].values[0], 1)
    expected_losses = round(wins_df[wins_df['team'] == team_name]['expected_loss'].values[0], 1)
    rank = team_data[team_data['team'] == team_name].index[0] + 1
    transposed_data = wins_df.transpose()[1:14].reset_index().reset_index()
    transposed_data.columns.values[2] = 'prob'
    win_6 = round(100 * wins_df['WIN6%'].values[0], 1)
    last_week_rating = last_week_data[last_week_data['team'] == team_name]['power_rating'].values[0]
    last_month_rating = last_month_data[last_month_data['team'] == team_name]['power_rating'].values[0]
    start_season_rating = start_season_data[start_season_data['team'] == team_name]['power_rating'].values[0]

    wins = records[records['team'] == team_name]['wins'].values[0]
    losses = records[records['team'] == team_name]['losses'].values[0]

    team_completed_games = completed_games[(completed_games['home_team'] == team_name) | (completed_games['away_team'] == team_name)]
    team_completed_games['team_win_prob'] = np.where(team_completed_games['home_team'] == team_name, 
                                    team_completed_games['PEAR_win_prob'], 
                                    100 - team_completed_games['PEAR_win_prob'])
    completed_expected_wins = round(sum(team_completed_games['team_win_prob']) / 100, 1)
    completed_expected_losses = round(len(team_completed_games) - completed_expected_wins, 1)
    games_played = wins + losses
    if len(team_completed_games) != games_played:
        completed_expected_wins = completed_expected_wins + 1

    win_out = wins + 12 - games_played
    win_out_percentage = round(wins_df[f'win_{win_out}'].values[0] * 100, 1)
    def find_team_spread(game, team_data, team_name, home_team, away_team, neutral):
        home_rating = team_data[team_data['team'] == home_team]['power_rating'].values[0]
        away_rating = team_data[team_data['team'] == away_team]['power_rating'].values[0]
        home_win_prob = game['home_win_prob']
        def adjust_home_pr(home_win_prob):
            return ((home_win_prob - 50) / 50) * 5
        def round_to_nearest_half(x):
            return np.round(x * 2) / 2 
        if neutral:
            spread = round((home_rating + adjust_home_pr(home_win_prob) - away_rating),1)
        else:
            spread = round((4.6 + home_rating + adjust_home_pr(home_win_prob) - away_rating),1)
        if (home_team == team_name) & (spread > 0):
            output = "-" + str(spread)
        elif (home_team == team_name) & (spread < 0):
            output = "+" + str(abs(spread))
        elif (home_team != team_name) & (spread < 0):
            output = str(spread)
        elif (spread == 0.0):
            output = str(spread)
        else:
            output = "+" + str(spread)

        return output, spread
    
    def add_spreads(schedule_info, team_data, team_name):
        # Define a helper function to apply the find_team_spread to each row
        def compute_spread(row):
            # Extract relevant data from the current row
            home_team = row['home_team']
            away_team = row['away_team']
            neutral = row['neutral']
            
            # Call the find_team_spread function with this row's data
            spread_str, raw_spread = find_team_spread(row, team_data, team_name, home_team, away_team, neutral)
            
            # Return both values as a tuple, so we can unpack them into two columns
            return pd.Series([spread_str, raw_spread])

        # Apply the compute_spread function to each row of the DataFrame
        schedule_info[['spread', 'raw_spread']] = schedule_info.apply(compute_spread, axis=1)
        
        return schedule_info

    # Example of calling the function
    schedule_info = add_spreads(schedule_info, team_data, team_name)

    ################################# BAR CHART #################################

    # Create the plot
    fig, (ax, ax2) = plt.subplots(nrows=2, figsize=(8, 12))
    fig.patch.set_facecolor('#CECEB2')  # Set figure background color
    ax.set_facecolor('#CECEB2')    
    team_logo_url = logos_df[logos_df['team'] == team_name]['logo'].values[0][0]
    team_color = logos_df[logos_df['team'] == team_name]['color'].values[0]
    
    # Plot the probabilities as a bar plot
    sns.barplot(x='level_0', y='prob', data=transposed_data, ax=ax, color=team_color)
    
    # Set titles and labels
    ax.set_title(f"#{rank} {team_name} Wins Distribution", fontsize=16, fontweight='bold')
    ax.set_xlabel("Wins", fontsize=12, fontweight='bold')
    ax.set_ylabel("Probability Based on 1000 Simulations", fontsize=12)

    ################################# TEAM LOGO BOTTOM MIDDLE #################################

    response = requests.get(team_logo_url)
    logo_img = Image.open(BytesIO(response.content))
    img_ax = fig.add_axes([1.42,0.1,0.55,0.55])
    img_ax.imshow(logo_img)
    img_ax.axis('off')

    ################################# AVERAGE TEAM STATISTICS #################################

    entire_team_schedule = entire_schedule[(entire_schedule['home_team'] == team_name) | (entire_schedule['away_team'] == team_name)].reset_index()
    average_pr = team_data['power_rating'].mean()
    num_simulations = 1000
    average_team_threshold = average_team_distribution(num_simulations, entire_team_schedule, average_pr, team_name)
    transposed_avg = average_team_threshold.reset_index()  # Reset index to create a new DataFrame
    transposed_avg = transposed_avg.melt(id_vars='index', var_name='win_count', value_name='prob')[0:13]  # Melt the DataFrame
    transposed_avg['win_count'] = transposed_avg['win_count'].str.replace('win_', '').astype(int) 
    transposed_avg = transposed_avg.drop(columns=['index'])
    team_completed_games['avg_win_prob'] = np.where(team_completed_games['home_team'] == team_name, 
                                    PEAR_Win_Prob(average_pr, team_completed_games['away_pr']), 
                                    100 - PEAR_Win_Prob(team_completed_games['home_pr'], average_pr))
    avg_expected_wins = round(sum(team_completed_games['avg_win_prob']) / 100, 1)
    avg_expected_loss = round(len(team_completed_games) - avg_expected_wins, 1)
    strength_of_record = SOR[SOR['team'] == team_name]['SOR'].values[0]
    strength_of_schedule = SOS[SOS['team'] == team_name]['SOS'].values[0]

    games_played = wins + losses
    if len(team_completed_games) != games_played:
        avg_expected_wins = avg_expected_wins + 1

    cmap = LinearSegmentedColormap.from_list('dark_gradient_orange', ['#660000', '#8B0000', '#CC5500', '#2C5E00', '#1D4D00'], N=100)
    # Function to get color based on value with gradient
    def get_color(value, vmin=0, vmax=100):
        norm_value = (value - vmin) / (vmax - vmin)  # Normalize the value between 0 and 1
        return cmap(norm_value)  # Get the color from the colormap
    
    xwin = LinearSegmentedColormap.from_list('dark_gradient_pattern', ['#660000', '#8B0000', '#CC5500', '#2C5E00', '#1D4D00'], N=100)

    # Function to get color based on value with specified boundaries
    def gradient_xWins(value, vmin=-2, vmax=2):
        if value <= vmin:
            return '#660000'  # Lighter dark red for <= -1.5
        elif value >= vmax:
            return '#1D4D00'  # Dark green for >= 1.5
        else:
            norm_value = (value - vmin) / (vmax - vmin)  # Normalize value between vmin and vmax
            return xwin(norm_value)  # Get color from colormap for intermediate values

    plt.text(2.45, 1.03, f"Schedule Stats", fontsize=12, ha='center', verticalalignment='top', transform=ax2.transAxes, fontweight='bold')
    plt.text(2.45, 0.98, f"Strength of Schedule: #{strength_of_schedule}", verticalalignment='top', ha='center', transform=ax2.transAxes, fontsize=12)
    plt.text(2.45, 0.93, f"Strength of Record: #{strength_of_record}", verticalalignment='top', ha='center', transform=ax2.transAxes, fontsize=12)
    plt.text(2.45, 0.88, f"Avg Team Current xRecord: {avg_expected_wins} - {avg_expected_loss}", ha='center', verticalalignment='top', transform=ax2.transAxes, fontsize=12)
    plt.text(2.45, 0.83, f"Avg Team Final xRecord: {round(average_team_threshold.loc[0, 'expected_wins'],1)} - {round(average_team_threshold.loc[0, 'expected_losses'], 1)}", ha='center', verticalalignment='top', transform=ax2.transAxes, fontsize=12)
    # plt.text(1.01, 0.78, f"Avg Team Win 6%: {round(average_team_threshold.loc[0, 'WIN6%']*100, 2)}%", verticalalignment='top', transform=ax2.transAxes)
    plt.text(2.45, 0.78, f"Wins Above Average: {round(wins - avg_expected_wins, 1)}", verticalalignment='top', ha='center', transform=ax2.transAxes, color = gradient_xWins(round(wins - avg_expected_wins, 1)), fontweight='bold', fontsize=12)
    plt.text(2.45, 0.73, f"Expected Wins Above Average: {round(completed_expected_wins - avg_expected_wins, 1)}", ha='center', fontsize=12, verticalalignment='top', transform=ax2.transAxes, color = gradient_xWins(round(completed_expected_wins - avg_expected_wins, 1)), fontweight='bold')
    plt.text(2.45, 0.68, f"Adjusted Performance: {round(wins - completed_expected_wins, 1)}", verticalalignment='top', ha='center', transform=ax2.transAxes, color=gradient_xWins(round(wins - completed_expected_wins, 1)), fontweight='bold', fontsize=12)
    plt.text(2.45, 0.63, f"Schedule-Adjusted Final Wins: {round(expected_wins - round(average_team_threshold.loc[0, 'expected_wins'],1), 1)}", ha='center',verticalalignment='top', transform=ax2.transAxes, color = gradient_xWins(round(expected_wins - round(average_team_threshold.loc[0, 'expected_wins'],1), 1)), fontweight='bold', fontsize=12)

    ################################# AVERAGE TEAM DISTRIBUTION CHART #################################

    fig.patch.set_facecolor('#CECEB2')  # Set figure background color
    ax2.set_facecolor('#CECEB2')    
    # Plot the probabilities as a bar plot
    sns.barplot(x='win_count', y='prob', data=transposed_avg, ax=ax2, color='#555555')
    
    # Set titles and labels
    ax2.set_title(f"Average Team vs. {team_name} Schedule", fontsize=16, fontweight='bold')
    ax2.set_xlabel("Wins", fontsize=12, fontweight='bold')
    ax2.set_ylabel("Probability Based on 1000 Simulations", fontsize=12)

    ################################# TEAM STATISTICS #################################

    cmap = LinearSegmentedColormap.from_list('dark_gradient_orange', ['#660000', '#8B0000', '#CC5500', '#2C5E00', '#1D4D00'], N=100)
    # Function to get color based on value with gradient
    def get_color(value, vmin=0, vmax=100):
        norm_value = (value - vmin) / (vmax - vmin)  # Normalize the value between 0 and 1
        return cmap(norm_value)  # Get the color from the colormap
    
    xwin = LinearSegmentedColormap.from_list('dark_gradient_pattern', ['#660000', '#8B0000', '#CC5500', '#2C5E00', '#1D4D00'], N=100)

    # Function to get color based on value with specified boundaries
    def get_gradient_color(value, vmin=-1.5, vmax=1.5):
        if value <= vmin:
            return '#660000'  # Lighter dark red for <= -1.5
        elif value >= vmax:
            return '#1D4D00'  # Dark green for >= 1.5
        else:
            norm_value = (value - vmin) / (vmax - vmin)  # Normalize value between vmin and vmax
            return xwin(norm_value)  # Get color from colormap for intermediate values

    # Fetch the scaled talent and add it as text
    power_rating = round(all_data[all_data['team'] == team_name]['power_rating'].values[0], 2)
    talent_scaled = round(all_data[all_data['team'] == team_name]['talent_scaled'].values[0], 2)
    offense_success = round(all_data[all_data['team'] == team_name]['offense_success_scaled'].values[0], 2)
    defense_success = round(all_data[all_data['team'] == team_name]['defense_success_scaled'].values[0], 2)
    offense_explosive = round(all_data[all_data['team'] == team_name]['offense_explosive'].values[0], 2)
    defense_explosive = round(all_data[all_data['team'] == team_name]['defense_explosive'].values[0], 2)
    turnovers = round(all_data[all_data['team'] == team_name]['total_turnovers_scaled'].values[0], 2)
    penalties = round(all_data[all_data['team'] == team_name]['penalties_scaled'].values[0], 2)

        ################################# CONFERENCE WINS INFO #################################

    conference_wins = records[records['team'] == team_name]['conference_wins'].values[0]
    conference_losses = records[records['team'] == team_name]['conference_losses'].values[0]
    expected_conference_wins = round(all_conference_wins[all_conference_wins['team'] == team_name]['expected_wins'].values[0], 1)
    expected_conference_losses = round(all_conference_wins[all_conference_wins['team'] == team_name]['expected_loss'].values[0], 1)
    conference = all_conference_wins[all_conference_wins['team'] == team_name]['conference'].values[0]

    if conference != 'FBS Independents':
        plt.text(1.01, 0.88, f"Current Record: {wins} - {losses} ({conference_wins} - {conference_losses})", verticalalignment='top', transform=ax.transAxes, fontweight='bold')
    else:
        plt.text(1.01, 0.88, f"Current Record: {wins} - {losses}", verticalalignment='top', transform=ax.transAxes, fontweight='bold')
    plt.text(1.01, 0.78, f"Final xRecord: {expected_wins} - {expected_losses}", verticalalignment='top', transform=ax.transAxes)
    plt.text(1.01, 0.83, f"Current xRecord: {completed_expected_wins} - {completed_expected_losses}", verticalalignment='top', transform=ax.transAxes)
    plt.text(1.01, 0.73, f"Win 6%: {win_6}%", fontsize=12, verticalalignment='top', transform=ax.transAxes)
    plt.text(1.01, 0.68, f"Win Out%: {win_out_percentage}%", fontsize=12, verticalalignment='top', transform=ax.transAxes)
    plt.text(1.01, 0.63, f"Team Talent: {talent_scaled}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(talent_scaled/10), fontweight='bold')
    plt.text(1.01, 0.58, f"Offense Success: {offense_success}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(offense_success), fontweight='bold')
    plt.text(1.01, 0.53, f"Offense Explosiveness: {offense_explosive}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(offense_explosive), fontweight='bold')
    plt.text(1.01, 0.48, f"Defense Success: {defense_success}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(defense_success), fontweight='bold')
    plt.text(1.01, 0.43, f"Defense Explosiveness: {defense_explosive}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(defense_explosive), fontweight='bold')
    plt.text(1.01, 0.38, f"Turnovers: {turnovers}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color = get_color(turnovers), fontweight='bold')
    plt.text(1.01, 0.33, f"Penalties: {penalties}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(penalties), fontweight='bold')

    ################################# NEXT GAME #################################    

    def get_watchability_color(value, vmin=-20, vmax=20):
        if value <= vmin:
            return '#660000'
        elif value >= vmax:
            return '#1D4D00'
        else:
            norm_value = (value - vmin) / (vmax - vmin)  # Normalize the value between 0 and 1
            return cmap(norm_value)  # Get the color from the colormap

    next_game = schedule_info[(schedule_info['home_team'] == team_name) | (schedule_info['away_team'] == team_name)].reset_index()[:1]
    if next_game['week'].values[0] != current_week:
        plt.text(1.01, 0.28, "This Week: BYE", fontsize = 12, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
    else:
        if next_game['home_team'].values[0] == team_name:
            opponent = next_game['away_team'].values[0]
        else:
            opponent = next_game['home_team'].values[0]

        spread = abs(next_game['raw_spread'].values[0])
        watchability = ((team_data[team_data['team'] == team_name]['power_rating'].values[0] + team_data[team_data['team'] == opponent]['power_rating'].values[0]) / 2) - (spread * 0.5) + 7
        opponent_url = logos[logos['team'] == opponent]['logo'].values[0][0]
        response = requests.get(opponent_url)
        logo_img = Image.open(BytesIO(response.content))
        img_ax = fig.add_axes([0.98, 0.53, 0.1, 0.1], anchor='NE', zorder=1)
        img_ax.imshow(logo_img)
        img_ax.axis('off')  # No axis for the image
        plt.text(1.01, 0.28, f"This Week", fontsize = 12, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
        opponent_rank = team_data[team_data['team'] == opponent].index[0] + 1
        plt.text(1.01, 0.23, f"#{opponent_rank} {opponent}", fontsize = 12, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
        plt.text(1.01, 0.04, f"Game Quality: {round(watchability,1)}", fontsize = 12, verticalalignment='top', transform=ax.transAxes, color=get_watchability_color(watchability),fontweight='bold')


    ################################# LIKELY LOSS #################################

    team_schedule = schedule_info[(schedule_info['home_team'] == team_name) | (schedule_info['away_team'] == team_name)].reset_index()
    team_schedule['raw_spread'] = pd.to_numeric(team_schedule['spread'], errors='coerce')
    team_schedule['team_win_prob'] = np.where(team_schedule['home_team'] == team_name, 
                                 team_schedule['PEAR_win_prob'], 
                                 100 - team_schedule['PEAR_win_prob'])
    lowest_prob_row = team_schedule.loc[team_schedule['team_win_prob'].idxmin()]
    if lowest_prob_row['home_team'] == team_name:
        most_likely_loss = lowest_prob_row['away_team']
    else:
        most_likely_loss = lowest_prob_row['home_team']
    likely_loss_rank = team_data[team_data['team'] == most_likely_loss].index[0] + 1
    loss_url = logos[logos['team'] == most_likely_loss]['logo'].values[0][0]
    response = requests.get(loss_url)
    loss_logo = Image.open(BytesIO(response.content))
    img_ax = fig.add_axes([1.32,.547,0.10,0.10])
    img_ax.imshow(loss_logo)
    img_ax.axis('off')
    # plt.text(1.335, 0.33, f"The Most Likely Loss, Win, and Closest Game Based on WP", fontsize = 12, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
    plt.text(1.45, 0.22, f"Most Likely Loss", fontsize = 12, verticalalignment='top', ha='center', transform=ax.transAxes, fontweight='bold')
    plt.text(1.45, 0.04, f"#{likely_loss_rank} {most_likely_loss}", fontsize = 12, ha='center', verticalalignment='top', transform=ax.transAxes, fontweight='bold')

    ################################# LIKELY WIN #################################

    highest_prob_row = team_schedule.loc[team_schedule['team_win_prob'].idxmax()]
    if highest_prob_row['home_team'] == team_name:
        most_likely_win = highest_prob_row['away_team']
    else:
        most_likely_win = highest_prob_row['home_team']
    likely_win_rank = team_data[team_data['team'] == most_likely_win].index[0] + 1
    win_url = logos[logos['team'] == most_likely_win]['logo'].values[0][0]
    response = requests.get(win_url)
    win_logo = Image.open(BytesIO(response.content))
    img_ax = fig.add_axes([1.58,.547,0.10,0.10])
    img_ax.imshow(win_logo)
    img_ax.axis('off')
    plt.text(1.75, 0.22, f"Most Likely Win", fontsize = 12, verticalalignment='top', ha='center',transform=ax.transAxes, fontweight='bold')
    plt.text(1.75, 0.04, f"#{likely_win_rank} {most_likely_win}", fontsize = 12, ha='center',verticalalignment='top', transform=ax.transAxes, fontweight='bold')

    ################################# CLOSEST GAME #################################

    closest_index = (team_schedule['team_win_prob']-50).abs().idxmin()
    closest_prob_row = team_schedule.loc[closest_index]
    if closest_prob_row['home_team'] == team_name:
        closest_game_team = closest_prob_row['away_team']
    else:
        closest_game_team = closest_prob_row['home_team']
    closest_rank = team_data[team_data['team'] == closest_game_team].index[0] + 1
    close_url = logos[logos['team'] == closest_game_team]['logo'].values[0][0]
    response = requests.get(close_url)
    close_logo = Image.open(BytesIO(response.content))
    img_ax = fig.add_axes([1.84,.547,0.10,0.10])
    img_ax.imshow(close_logo)
    img_ax.axis('off')
    plt.text(2.05, 0.22, f"Closest Game", fontsize = 12, verticalalignment='top', ha='center',transform=ax.transAxes, fontweight='bold')
    plt.text(2.05, 0.04, f"#{closest_rank} {closest_game_team}", fontsize = 12, ha='center',verticalalignment='top', transform=ax.transAxes, fontweight='bold')

    ################################# SCHEDULE INFO #################################    
    
    def get_color_wl(w_l):
        if w_l == "W":
            return '#1D4D00'
        else:
            return '#660000'
    j = 0.83
    plt.text(1.52, 0.88, f"FCS Games are Not Included in Schedule", fontsize = 12, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
    for i, game in team_completed_games.iterrows():
        neutral = game['neutral']
        plt.text(1.36, j, f"{game['week']}", fontsize = 13, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
        if game['home_team'] == team_name:
            if game['home_points'] > game['away_points']:
                w_l = 'W'
            else:
                w_l = 'L'
        else:
            if game['home_points'] > game['away_points']:
                w_l = 'L'
            else:
                w_l = 'W'
        if neutral:
            if game['home_team'] == team_name:
                opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                plt.text(1.4, j, f"{game['home_team']} (N) {game['away_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')
            else:
                opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                plt.text(1.4, j, f"{game['away_team']} (N) {game['home_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')       
        elif game['home_team'] == team_name:
            opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
            plt.text(1.4, j, f"{game['home_team']} vs. {game['away_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')
        else:
            opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
            plt.text(1.4, j, f"{game['away_team']} @ {game['home_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')
        plt.text(2.02, j, f"{w_l}", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')
        if game['home_points'] > game['away_points']:
            plt.text(2.12, j, f"{int(game['home_points'])}-{int(game['away_points'])}", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')
        else:
            plt.text(2.12, j, f"{int(game['away_points'])}-{int(game['home_points'])}", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color = get_color_wl(w_l), fontweight='bold')
        j -= 0.05

    def get_color_future(w_l):
        if w_l <= 0:
            return '#1D4D00'
        else:
            return '#660000'
    for i, game in team_schedule.iterrows():
        neutral = game['neutral']
        plt.text(1.36, j, f"{game['week']}", fontsize = 13, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
        spread = game['spread']
        raw_spread = game['raw_spread']
        if neutral:
            if game['home_team'] == team_name:
                opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
                plt.text(1.4, j, f"{game['home_team']} (N) {game['away_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes)
            else:
                opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
                plt.text(1.4, j, f"{game['away_team']} (N) {game['home_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes)
        elif game['home_team'] == team_name:
            opponent_rank = team_data[team_data['team'] == game['away_team']].index[0] + 1
            plt.text(1.4, j, f"{game['home_team']} vs. {game['away_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes)
        else:
            opponent_rank = team_data[team_data['team'] == game['home_team']].index[0] + 1
            plt.text(1.4, j, f"{game['away_team']} @ {game['home_team']} (#{opponent_rank})", fontsize = 13, verticalalignment='top', transform=ax.transAxes)
        plt.text(2.02, j, f"{round(game['team_win_prob'],1)}%", fontsize = 13, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
        plt.text(2.12, j, f"{spread}", fontsize = 13, verticalalignment='top', transform=ax.transAxes, color=get_color_future(raw_spread), fontweight='bold')
        # plt.text(1.75, j, f"{int(game['home_points'])}-{int(game['away_points'])}", fontsize = 12, verticalalignment='top', transform=ax.transAxes)
        j -= 0.05

    ################################# CHANCE TO WIN _ GAMES #################################

    import matplotlib.font_manager as fm
    checkmark_font = fm.FontProperties(family='DejaVu Sans')
    import matplotlib.colors as mcolors

    dark_green = '#1D4D00'
    medium_green = '#3C7300'
    orange = '#D2691E'
    black = 'black'
    def interpolate_color(c1, c2, factor):
        """Interpolate between two colors c1 and c2 based on factor (0 to 1)."""
        color1 = mcolors.hex2color(c1)
        color2 = mcolors.hex2color(c2)
        return mcolors.rgb2hex([(1 - factor) * a + factor * b for a, b in zip(color1, color2)])
    def get_color_percentage(percentage):
        if 40 <= percentage <= 60:
            return dark_green  # Dark green is constant in this range
        elif 30 <= percentage < 40:
            factor = (percentage - 30) / (40 - 30)
            return interpolate_color(orange, medium_green, factor)
        elif 60 < percentage <= 70:
            factor = (percentage - 60) / (70 - 60)
            return interpolate_color(medium_green, orange, factor)
        elif 15 <= percentage < 30:
            factor = (percentage - 15) / (30 - 15)
            return interpolate_color(black, orange, factor)
        elif 70 < percentage <= 85:
            factor = (percentage - 70) / (85 - 70)
            return interpolate_color(orange, black, factor)
        else:
            return black  # Anything outside the defined ranges is black

    plt.text(2.32, 0.88, "Overall xWins", fontsize=12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
    plt.text(2.24, 0.83, "Wins", fontsize=12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
    plt.text(2.32, 0.83, ">=", fontsize = 12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
    plt.text(2.40, 0.83, "=", fontsize = 12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
    team_index = wins_df[wins_df['team'] == team_name].index[0]
    win_columns = [f'win_{j}' for j in range(13)]
    cumulative_probs = wins_df.loc[team_index, win_columns].values[::-1].cumsum()[::] * 100

    for j in range(len(cumulative_probs)):
        if 12-j == wins_df[wins_df['team'] == team_name]['projected_wins'].values[0]:
            plt.text(2.24, 0.78 - 0.05*j, f"{12-j}", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color = 'darkblue')
        elif j == 12-wins:
            plt.text(2.24, 0.78 - 0.05*j, f"{12-j}", fontsize=12, transform=ax.transAxes, verticalalignment='top', fontweight='bold', fontproperties=checkmark_font, color='green', ha='center')
        else:
            plt.text(2.24, 0.78 - 0.05*j, f"{12-j}", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold')
        if j > 12-wins:
            continue
        else:
            if cumulative_probs[j] == 0:
                continue
            elif cumulative_probs[j] < 1:
                plt.text(2.32, 0.78 - 0.05*j, f"<1%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(1))
            elif cumulative_probs[j] == 100:
                plt.text(2.32, 0.78 - 0.05*j, f"100%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(99))   
            elif cumulative_probs[j] >= 99:
                plt.text(2.32, 0.78 - 0.05*j, f">99%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(99))                
            else:
                plt.text(2.32, 0.78 - 0.05*j, f"{round(cumulative_probs[j])}%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(cumulative_probs[j]))

    for i in range(len(win_columns[::-1])):
        if i > 12-wins:
            continue
        else:
            if wins_df.loc[team_index, win_columns[::-1][i]]*100 == 0:
                continue
            elif wins_df.loc[team_index, win_columns[::-1][i]]*100 <= 1:
                plt.text(2.40, 0.78 - 0.05*i, f"<1%", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color=get_color_percentage(1))
            elif wins_df.loc[team_index, win_columns[::-1][i]]*100 == 100:
                plt.text(2.40, 0.78 - 0.05*i, f"100%", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color=get_color_percentage(99))
            elif wins_df.loc[team_index, win_columns[::-1][i]]*100 >= 99:
                plt.text(2.40, 0.78 - 0.05*i, f">99%", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color=get_color_percentage(99))
            else:
                plt.text(2.40, 0.78 - 0.05*i, f"{round(wins_df.loc[team_index, win_columns[::-1][i]]*100)}%", fontsize=12, fontweight='bold', transform=ax.transAxes, verticalalignment='top', ha='center', color = get_color_percentage(wins_df.loc[team_index, win_columns[::-1][i]]*100))

    num_conference_games = {
        'SEC':8,
        'Big Ten':9,
        'ACC':8,
        'Big 12':9,
        'Mountain West':7,
        'American Athletic':9,
        'Pac-12':1,
        'Sun Belt':8,
        'Mid-American':8,
        'Conference USA':8
    }

    team_conference = all_conference_wins[all_conference_wins['team'] == team_name]['conference'].values[0]
    if team_conference != 'FBS Independents':     
        plt.text(2.56, 0.88, "Conference xWins", fontsize=12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
        plt.text(2.48, 0.83, "Wins", fontsize=12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
        plt.text(2.56, 0.83, ">=", fontsize = 12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
        plt.text(2.64, 0.83, "=", fontsize = 12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center')
        team_index = all_conference_wins[all_conference_wins['team'] == team_name].index[0]
        this_conference_wins = all_conference_wins[all_conference_wins['conference'] == team_conference]
        conference_games = num_conference_games[team_conference]  
        win_columns = [f'win_{j}' for j in range(conference_games+1)]
        cumulative_probs = this_conference_wins.loc[team_index, win_columns].values[::-1].cumsum()[::] * 100
        for j in range(0,conference_games+1):
            if conference_games-j == all_conference_wins[all_conference_wins['team'] == team_name]['projected_wins'].values[0]:
                plt.text(2.48, 0.78 - 0.05*j, f"{conference_games-j}", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color = 'darkblue')
            elif j == conference_games-conference_wins:
                plt.text(2.48, 0.78 - 0.05*j, f"{conference_games-j}", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', fontproperties=checkmark_font, color='green', ha='center')
            else:
                plt.text(2.48, 0.78 - 0.05*j, f"{conference_games-j}", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold')
            if j > conference_games-conference_wins:
                continue
            else:
                if cumulative_probs[j] == 0:
                    continue
                elif cumulative_probs[j] < 1:
                    plt.text(2.56, 0.78 - 0.05*j, f"<1%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(1))
                elif cumulative_probs[j] == 100:
                    plt.text(2.56, 0.78 - 0.05*j, f"100%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(100))
                elif cumulative_probs[j] >= 99:
                    plt.text(2.56, 0.78 - 0.05*j, f">99%", fontsize=12, transform=ax.transAxes,  fontweight='bold',verticalalignment='top', ha='center', color=get_color_percentage(99))
                else:
                    plt.text(2.56, 0.78 - 0.05*j, f"{round(cumulative_probs[j])}%", fontsize=12, transform=ax.transAxes, fontweight='bold', verticalalignment='top', ha='center',color=get_color_percentage(cumulative_probs[j]))

        for i in range(len(win_columns[::-1])):
            if i > conference_games - conference_wins:
                continue
            else:
                if this_conference_wins.loc[team_index, win_columns[::-1][i]]*100 == 0:
                    continue
                elif this_conference_wins.loc[team_index, win_columns[::-1][i]]*100 <= 1:
                    plt.text(2.64, 0.78 - 0.05*i, f"<1%", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color=get_color_percentage(1))
                elif this_conference_wins.loc[team_index, win_columns[::-1][i]]*100 == 100:
                    plt.text(2.64, 0.78 - 0.05*i, f"100%", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color=get_color_percentage(99)) 
                elif this_conference_wins.loc[team_index, win_columns[::-1][i]]*100 >= 99:
                    plt.text(2.64, 0.78 - 0.05*i, f">99%", fontsize=12, transform=ax.transAxes, verticalalignment='top', ha='center', fontweight='bold', color=get_color_percentage(99))                    
                else:
                    plt.text(2.64, 0.78 - 0.05*i, f"{round(this_conference_wins.loc[team_index, win_columns[::-1][i]]*100)}%", fontsize=12, fontweight='bold', transform=ax.transAxes, verticalalignment='top', ha='center', color=get_color_percentage(this_conference_wins.loc[team_index, win_columns[::-1][i]]*100))

    ################################# NEXT OPPONENT #################################

    plt.text(1.01, -0.01, "Opponent's Rankings",fontsize = 12, verticalalignment='top', transform=ax.transAxes, fontweight='bold')
    if next_game['week'].values[0] == current_week:
        opp_power_rating = round(all_data[all_data['team'] == opponent]['power_rating'].values[0], 2)
        opp_talent_scaled = round(all_data[all_data['team'] == opponent]['talent_scaled'].values[0], 2)
        opp_offense_success = round(all_data[all_data['team'] == opponent]['offense_success_scaled'].values[0], 2)
        opp_defense_success = round(all_data[all_data['team'] == opponent]['defense_success_scaled'].values[0], 2)
        opp_offense_explosive = round(all_data[all_data['team'] == opponent]['offense_explosive'].values[0], 2)
        opp_defense_explosive = round(all_data[all_data['team'] == opponent]['defense_explosive'].values[0], 2)
        opp_turnovers = round(all_data[all_data['team'] == opponent]['total_turnovers_scaled'].values[0], 2)
        opp_penalties = round(all_data[all_data['team'] == opponent]['penalties_scaled'].values[0], 2)

        plt.text(1.01, -0.06, f"Opponent RTG: {opp_power_rating}", fontsize=12, verticalalignment='top', transform=ax.transAxes)
        plt.text(1.01, -0.11, f"Team Talent: {opp_talent_scaled}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(opp_talent_scaled/10), fontweight='bold')
        plt.text(1.01, -0.16, f"Offense Success: {opp_offense_success}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(opp_offense_success), fontweight='bold')
        plt.text(1.01, -0.21, f"Offense Explosiveness: {opp_offense_explosive}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(opp_offense_explosive), fontweight='bold')
        plt.text(1.01, -0.26, f"Defense Success: {opp_defense_success}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(opp_defense_success), fontweight='bold')
        plt.text(1.01, -0.31, f"Defense Explosiveness: {opp_defense_explosive}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(opp_defense_explosive), fontweight='bold')
        plt.text(1.01, -0.36, f"Turnovers: {opp_turnovers}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color = get_color(opp_turnovers), fontweight='bold')
        plt.text(1.01, -0.41, f"Penalties: {opp_penalties}", fontsize=12, verticalalignment='top', transform=ax.transAxes, color=get_color(opp_penalties), fontweight='bold')

    ################################# ENDING BOILER PLATE #################################
    if branding:
        plt.text(1.82, .14, "@PEARRatingCFB", fontsize=16, ha='center', verticalalignment='top', transform=ax2.transAxes, fontweight='bold')
        img_path = './PEAR_logo.jpg'  # Replace with the actual path to the image
        img = mpimg.imread(img_path)
        img_ax = fig.add_axes([2.1,.04,0.30,0.30])
        img_ax.imshow(img)
        img_ax.axis('off')
    
    plt.text(1.82, 0.32, f"RTG: {power_rating} (#{rank})", fontsize=16, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', ha='center')
    if power_rating >= last_week_rating:
        plt.text(1.82, 0.27, f"Week Change: +{round(power_rating-last_week_rating,1)}", fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', color = '#1D4D00', ha='center')
    if power_rating < last_week_rating:
        plt.text(1.82, 0.27, f"Week Change: {round(power_rating-last_week_rating,1)}", fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', color = '#660000', ha='center')

    if power_rating >= last_month_rating:
        plt.text(1.82, 0.23, f"Month Change: +{round(power_rating-last_month_rating,1)}", fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', color = '#1D4D00', ha='center')
    if power_rating < last_month_rating:
        plt.text(1.82, 0.23, f"Month Change: {round(power_rating-last_month_rating,1)}", fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', color = '#660000', ha='center')

    if power_rating >= start_season_rating:
        plt.text(1.82, 0.19, f"Season Change: +{round(power_rating-start_season_rating,1)}", fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', color = '#1D4D00', ha='center')
    if power_rating < start_season_rating:
        plt.text(1.82, 0.19, f"Season Change: {round(power_rating-start_season_rating,1)}", fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold', color = '#660000', ha='center')

    plt.text(1.01, 0.485, 'Data Dictionary', fontsize=12, verticalalignment='top', transform=ax2.transAxes, fontweight='bold')
    # plt.text(1.01, 0.47, "'x' stands for 'Expected'", fontsize=12, verticalalignment='top', transform=ax2.transAxes)
    # plt.text(1.01, 0.42, "'Avg' stands for 'Average'", fontsize=12, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.45, "0.0 is the average FBS Power Rating", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.42, "Team Stats are scaled from 1 to 100", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.39, "Team Talent is scaled from 100 to 1000", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.36, "Game Quality is between -27 and 27", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.33, "Wins Above Average:", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.30, "Wins compared to the average team", fontsize=8, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.27, "Expected Wins Above Average:", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.24, "xPerformance versus the average xPerformance", fontsize=8, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.21, "Adjusted Performance:", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.18, "Performance adjusted for schedule difficulty", fontsize=8, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.15, "Schedule-Adjusted Final Wins:", fontsize=10, verticalalignment='top', transform=ax2.transAxes)
    plt.text(1.01, 0.12, "Expected win total versus the average expected win total", fontsize=8, verticalalignment='top', transform=ax2.transAxes)

    # Improve the layout
    plt.tight_layout()
    plt.show()

## Actual Visuals Used